In [5]:
import pandas as pd, csv, textwrap
from pathlib import Path
from collections import defaultdict, deque

# -----------------------------
# 1) Ejemplo de insumo_base
#    Estructura: tabla_origen, tabla_destino, query
# -----------------------------
insumo_rows = [
    ("s_core.clientes", "proceso_bipa_vpr.stg_clientes",
     "create table proceso_bipa_vpr.stg_clientes as select * from s_core.clientes;"),
    ("s_core.prestamos", "proceso_bipa_vpr.stg_prestamos",
     "create table proceso_bipa_vpr.stg_prestamos as select * from s_core.prestamos;"),
    ("proceso_bipa_vpr.stg_clientes", "proceso_bipa_vpr.dim_clientes",
     "create table proceso_bipa_vpr.dim_clientes as select ... from proceso_bipa_vpr.stg_clientes;"),
    ("proceso_bipa_vpr.stg_prestamos", "proceso_bipa_vpr.fact_prestamos_stg",
     "create table proceso_bipa_vpr.fact_prestamos_stg as select ... from proceso_bipa_vpr.stg_prestamos;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.fact_prestamos_stg", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),

    ("s_core.pagos", "proceso_bipa_vpr.stg_pagos",
     "create table proceso_bipa_vpr.stg_pagos as select * from s_core.pagos;"),
    ("proceso_bipa_vpr.stg_pagos", "proceso_bipa_vpr.fact_pagos_stg",
     "create table proceso_bipa_vpr.fact_pagos_stg as select ... from proceso_bipa_vpr.stg_pagos;"),
    ("proceso_bipa_vpr.fact_pagos_stg", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_stg;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),

    ("resultados_bipa_vpr.tb_fact_prestamos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_prestamos;"),
    ("resultados_bipa_vpr.tb_fact_pagos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_pagos;"),

    ("s_core.cartera", "proceso_riesgo.stg_cartera",
     "create table proceso_riesgo.stg_cartera as select * from s_core.cartera;"),
    ("proceso_riesgo.stg_cartera", "proceso_riesgo.mora_clientes",
     "create table proceso_riesgo.mora_clientes as select ... from proceso_riesgo.stg_cartera;"),
    ("proceso_riesgo.mora_clientes", "resultados_riesgo.tb_alertas_mora",
     "create table resultados_riesgo.tb_alertas_mora as select ... from proceso_riesgo.mora_clientes;"),
]
insumo_base = pd.DataFrame(insumo_rows, columns=["tabla_origen","tabla_destino","query"])



In [3]:
# -----------------------------
# 2) Utilidades: zona, color e IDs
# -----------------------------
def parse_schema(fqn: str):
    fqn = (fqn or "").strip().lower()
    if "." in fqn:
        return fqn.split(".",1)[0]
    return fqn

def zone_prefix(fqn: str):
    schema = parse_schema(fqn)
    if schema.startswith("s_"):
        return "S"
    if schema.startswith("proceso"):
        return "P"
    if schema.startswith("resultados"):
        return "R"
    return "U"

def zone_color(fqn: str):
    z = zone_prefix(fqn)
    if z == "S":
        return "#e74c3c"
    if z == "P":
        return "#f1c40f"
    if z == "R":
        return "#2ecc71"
    return "#95a5a6"

def svg_escape(s: str):
    return (str(s)
            .replace("&","&amp;")
            .replace("<","&lt;")
            .replace(">","&gt;")
            .replace('"',"&quot;"))


def upstream_closure(target: str):
    """Retorna (nodos, aristas) del upstream completo del target."""
    target = target.strip()
    visited = set([target])
    q = deque([target])
    edges = []
    while q:
        d = q.popleft()
        for s, qry in incoming.get(d, []):
            edges.append((s, d, qry))
            if s not in visited:
                visited.add(s)
                q.append(s)
    return visited, edges

def assign_ids_per_objective(nodes, target):
    """
    IDs reinician por objetivo.
    Secuencia separada por zona: S1.., P1.., R1.. (y U1.. si aplica).
    Orden determinístico: más upstream primero, luego alfabético.
    """
    _, edges = upstream_closure(target)
    level = {target: 0}
    for _ in range(80):
        changed = False
        for s, d, _ in edges:
            if d in level:
                cand = level[d] + 1
                if s not in level or cand > level[s]:
                    level[s] = cand
                    changed = True
        if not changed:
            break

    def sort_key(n):
        return (-level.get(n, -1), n.lower())

    ordered = sorted(nodes, key=sort_key)

    counters = {"S":1, "P":1, "R":1, "U":1}
    node_id = {}
    for n in ordered:
        z = zone_prefix(n)
        node_id[n] = f"{z}{counters[z]}"
        counters[z] += 1
    return node_id

def build_svg_for_target(target, nodes, edges, node_id_map):
    # Layout jerárquico por niveles para no sobreponer nodos.
    level = {target: 0}
    for _ in range(120):
        changed = False
        for s, d, _ in edges:
            if d in level:
                cand = level[d] + 1
                if s not in level or cand > level[s]:
                    level[s] = cand
                    changed = True
        if not changed:
            break
    max_level = max(level.values()) if level else 0

    dx = 280
    node_w, node_h = 255, 70
    x_level = {n: (max_level - level.get(n, max_level))*dx for n in nodes}

    buckets = defaultdict(list)
    for n in nodes:
        buckets[level.get(n, max_level)].append(n)
    for k in buckets:
        buckets[k].sort(key=lambda n: n.lower())

    y_pos = {}
    for k, ns in buckets.items():
        for i, n in enumerate(ns):
            y_pos[n] = 60 + i*130

    width = (max_level+1)*dx + 420
    height = (max(y_pos.values()) if y_pos else 0) + 220

    parts = []
    parts.append('<div style="width:100%;height:100%;overflow:auto;background:#ffffff;">')
    parts.append(f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}" style="background:#ffffff;">')
    parts.append("""
<defs>
  <marker id="arrow" viewBox="0 0 10 10" refX="10" refY="5" markerWidth="7" markerHeight="7" orient="auto-start-reverse">
    <path d="M 0 0 L 10 5 L 0 10 z" fill="#555"></path>
  </marker>
</defs>
""".strip())

    # edges
    for s, d, _ in edges:
        x1 = x_level[s] + node_w
        y1 = y_pos[s] + node_h/2
        x2 = x_level[d]
        y2 = y_pos[d] + node_h/2
        mx = (x1 + x2)/2
        parts.append(f'<path d="M {x1} {y1} C {mx} {y1}, {mx} {y2}, {x2} {y2}" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrow)"/>')

    # nodes
    for n in sorted(nodes, key=lambda n: (x_level[n], y_pos[n])):
        x, y = x_level[n], y_pos[n]
        color = zone_color(n)
        nid = node_id_map.get(n, "")
        parts.append(f'<rect x="{x}" y="{y}" rx="10" ry="10" width="{node_w}" height="{node_h}" fill="{color}" stroke="#333" stroke-width="1.2"/>')
        parts.append(f'<text x="{x+12}" y="{y+26}" font-size="14" fill="#111" font-weight="700">{svg_escape(nid)}</text>')
        lines = textwrap.wrap(svg_escape(n), width=34)
        for j, line in enumerate(lines[:2]):
            parts.append(f'<text x="{x+12}" y="{y+46 + j*16}" font-size="12" fill="#111">{line}</text>')

    parts.append("</svg></div>")
    return "\n".join(parts)

In [9]:
# -----------------------------
# 3) Índice dst -> fuentes (para recorrer upstream)
# -----------------------------
incoming = defaultdict(list)
for _, r in insumo_base.iterrows():
    incoming[r["tabla_destino"].strip()].append((r["tabla_origen"].strip(), r["query"]))


# -----------------------------
# 4) Generar tb_linaje y tb_html para cada tabla_destino objetivo
# -----------------------------
tb_linaje_rows = []
tb_html_rows = []

targets = sorted(insumo_base["tabla_destino"].unique(), key=lambda s: s.lower())

for target in targets:
    nodes, edges = upstream_closure(target)
    node_id_map = assign_ids_per_objective(nodes, target)

    # tb_linaje: una fila por relación origen->destino; id = id del nodo destino
    for s, d, qry in edges:
        tb_linaje_rows.append({
            "tabla_linaje_objetivo": target,
            "id": node_id_map.get(d, ""),
            "tabla_origen": s,
            "tabla_destino": d,
            "query": qry
        })

    tb_html_rows.append({
        "tabla_destino": target,
        "html": build_svg_for_target(target, nodes, edges, node_id_map)
    })

tb_linaje = pd.DataFrame(tb_linaje_rows)
tb_html = pd.DataFrame(tb_html_rows)

# -----------------------------
# 5) Guardar CSVs
# -----------------------------
out_dir = "./resultados_linaje"
insumo_path = out_dir + "/insumo_base_ejemplo.csv"
linaje_path = out_dir +"/tb_linaje_generado.csv"
html_path = out_dir + "/tb_html_generado.csv"

insumo_base.to_csv(insumo_path, index=False, quoting=csv.QUOTE_ALL)
tb_linaje.to_csv(linaje_path, index=False, quoting=csv.QUOTE_ALL)
tb_html.to_csv(html_path, index=False, quoting=csv.QUOTE_ALL)

# Mostrar vistas rápidas
insumo_base_head = insumo_base.head(8)
tb_linaje_sample = tb_linaje.sort_values(["tabla_linaje_objetivo","tabla_destino","tabla_origen"]).head(15)
tb_html_list = tb_html[["tabla_destino"]].copy()

(insumo_base_head, tb_linaje_sample, tb_html_list, str(insumo_path), str(linaje_path), str(html_path))

(                          tabla_origen                          tabla_destino  \
 0                      s_core.clientes          proceso_bipa_vpr.stg_clientes   
 1                     s_core.prestamos         proceso_bipa_vpr.stg_prestamos   
 2        proceso_bipa_vpr.stg_clientes          proceso_bipa_vpr.dim_clientes   
 3       proceso_bipa_vpr.stg_prestamos    proceso_bipa_vpr.fact_prestamos_stg   
 4        proceso_bipa_vpr.dim_clientes  resultados_bipa_vpr.tb_fact_prestamos   
 5  proceso_bipa_vpr.fact_prestamos_stg  resultados_bipa_vpr.tb_fact_prestamos   
 6                         s_core.pagos             proceso_bipa_vpr.stg_pagos   
 7           proceso_bipa_vpr.stg_pagos        proceso_bipa_vpr.fact_pagos_stg   
 
                                                query  
 0  create table proceso_bipa_vpr.stg_clientes as ...  
 1  create table proceso_bipa_vpr.stg_prestamos as...  
 2  create table proceso_bipa_vpr.dim_clientes as ...  
 3  create table proceso_bipa_vpr.fa

In [20]:
import pandas as pd, csv, textwrap
from pathlib import Path
from collections import defaultdict, deque

# -----------------------------
# Ejemplo de insumo_base (igual que antes)
# -----------------------------
insumo_rows = [
    ("s_core.clientes", "proceso_bipa_vpr.stg_clientes",
     "create table proceso_bipa_vpr.stg_clientes as select * from s_core.clientes;"),
    ("s_core.prestamos", "proceso_bipa_vpr.stg_prestamos",
     "create table proceso_bipa_vpr.stg_prestamos as select * from s_core.prestamos;"),
    ("proceso_bipa_vpr.stg_clientes", "proceso_bipa_vpr.dim_clientes",
     "create table proceso_bipa_vpr.dim_clientes as select ... from proceso_bipa_vpr.stg_clientes;"),
    ("proceso_bipa_vpr.stg_prestamos", "proceso_bipa_vpr.fact_prestamos_stg",
     "create table proceso_bipa_vpr.fact_prestamos_stg as select ... from proceso_bipa_vpr.stg_prestamos;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.fact_prestamos_stg", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),

    ("s_core.pagos", "proceso_bipa_vpr.stg_pagos",
     "create table proceso_bipa_vpr.stg_pagos as select * from s_core.pagos;"),
    ("proceso_bipa_vpr.stg_pagos", "proceso_bipa_vpr.fact_pagos_stg",
     "create table proceso_bipa_vpr.fact_pagos_stg as select ... from proceso_bipa_vpr.stg_pagos;"),
    ("proceso_bipa_vpr.fact_pagos_stg", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_stg;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),

    ("resultados_bipa_vpr.tb_fact_prestamos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_prestamos;"),
    ("resultados_bipa_vpr.tb_fact_pagos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_pagos;"),

    ("s_core.cartera", "proceso_riesgo.stg_cartera",
     "create table proceso_riesgo.stg_cartera as select * from s_core.cartera;"),
    ("proceso_riesgo.stg_cartera", "proceso_riesgo.mora_clientes",
     "create table proceso_riesgo.mora_clientes as select ... from proceso_riesgo.stg_cartera;"),
    ("proceso_riesgo.mora_clientes", "resultados_riesgo.tb_alertas_mora",
     "create table resultados_riesgo.tb_alertas_mora as select ... from proceso_riesgo.mora_clientes;"),
]
insumo_base = pd.DataFrame(insumo_rows, columns=["tabla_origen","tabla_destino","query"])

# -----------------------------
# Utilidades
# -----------------------------
def parse_schema(fqn: str):
    fqn = (fqn or "").strip().lower()
    return fqn.split(".", 1)[0] if "." in fqn else fqn

def zone_prefix(fqn: str):
    schema = parse_schema(fqn)
    if schema.startswith("s_"):
        return "S"
    if schema.startswith("proceso"):
        return "P"
    if schema.startswith("resultados") or schema.startswith("resultado"):
        return "R"
    return "U"

def zone_color(fqn: str):
    z = zone_prefix(fqn)
    if z == "S":
        return "#e74c3c"
    if z == "P":
        return "#f1c40f"
    if z == "R":
        return "#2ecc71"
    return "#95a5a6"

def svg_escape(s: str):
    return (str(s)
            .replace("&","&amp;")
            .replace("<","&lt;")
            .replace(">","&gt;")
            .replace('"',"&quot;"))

# -----------------------------
# Construir índices
# -----------------------------
incoming = defaultdict(list)  # dst -> [(src, query)]
outgoing = defaultdict(list)  # src -> [(dst, query)]
for _, r in insumo_base.iterrows():
    s = r["tabla_origen"].strip()
    d = r["tabla_destino"].strip()
    incoming[d].append((s, r["query"]))
    outgoing[s].append((d, r["query"]))

# -----------------------------
# Closures
# -----------------------------
def upstream_closure(target: str):
    target = target.strip()
    visited = set([target])
    q = deque([target])
    edges = []
    while q:
        d = q.popleft()
        for s, qry in incoming.get(d, []):
            edges.append((s, d, qry))
            if s not in visited:
                visited.add(s)
                q.append(s)
    return visited, edges

def downstream_closure(root: str):
    root = root.strip()
    visited = set([root])
    q = deque([root])
    edges = []
    while q:
        s = q.popleft()
        for d, qry in outgoing.get(s, []):
            edges.append((s, d, qry))
            if d not in visited:
                visited.add(d)
                q.append(d)
    return visited, edges

# -----------------------------
# Levels para layout
# -----------------------------
def levels_upstream(edges, target):
    level = {target: 0}
    for _ in range(200):
        changed = False
        for s, d, _ in edges:
            if d in level:
                cand = level[d] + 1
                if s not in level or cand > level[s]:
                    level[s] = cand
                    changed = True
        if not changed:
            break
    return level

def levels_downstream(edges, root):
    level = {root: 0}
    for _ in range(200):
        changed = False
        for s, d, _ in edges:
            if s in level:
                cand = level[s] + 1
                if d not in level or cand > level[d]:
                    level[d] = cand
                    changed = True
        if not changed:
            break
    return level

# -----------------------------
# IDs (reinicio por objetivo)
# -----------------------------
def assign_ids_per_objective(nodes, level_map, kind):
    def sort_key(n):
        lv = level_map.get(n, -1)
        return ((-lv, n.lower()) if kind == "upstream" else (lv, n.lower()))
    ordered = sorted(nodes, key=sort_key)

    counters = {"S":1, "P":1, "R":1, "U":1}
    node_id = {}
    for n in ordered:
        z = zone_prefix(n)
        node_id[n] = f"{z}{counters[z]}"
        counters[z] += 1
    return node_id

# -----------------------------
# SVG/HTML (con scrollbars "fijos" al borde del visual)
# -----------------------------
def build_svg_html(nodes, edges, level_map, kind, node_ids):
    dx = 350  # más separación horizontal
    dy = 140
    node_w, node_h = 270, 74

    max_level = max(level_map.values()) if level_map else 0

    if kind == "upstream":
        x_level = {n: (max_level - level_map.get(n, max_level)) * dx for n in nodes}
    else:
        x_level = {n: level_map.get(n, 0) * dx for n in nodes}

    buckets = defaultdict(list)
    for n in nodes:
        buckets[level_map.get(n, 0)].append(n)
    for k in buckets:
        buckets[k].sort(key=lambda n: n.lower())

    y_pos = {}
    for k, ns in buckets.items():
        for i, n in enumerate(ns):
            y_pos[n] = 60 + i * dy

    width = (max_level + 1) * dx + 520
    height = (max(y_pos.values()) if y_pos else 0) + 260

    parts = []
    parts.append('<div style="position:relative;width:100%;height:320px;background:#ffffff;">')
    parts.append('<div style="position:absolute;inset:0;overflow-x:scroll;overflow-y:scroll;background:#ffffff;">')
    parts.append(f'<div style="width:{width}px;height:{height}px;">')
    parts.append(f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}" style="background:#ffffff;">')

    parts.append("""
<defs>
  <marker id="arrow" viewBox="0 0 10 10" refX="10" refY="5" markerWidth="7" markerHeight="7" orient="auto-start-reverse">
    <path d="M 0 0 L 10 5 L 0 10 z" fill="#555"></path>
  </marker>
</defs>
""".strip())

    # edges
    for s, d, _ in edges:
        if s not in x_level or d not in x_level:
            continue
        x1 = x_level[s] + node_w
        y1 = y_pos[s] + node_h/2
        x2 = x_level[d]
        y2 = y_pos[d] + node_h/2
        mx = (x1 + x2) / 2
        parts.append(
            f'<path d="M {x1} {y1} C {mx} {y1}, {mx} {y2}, {x2} {y2}" '
            f'stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrow)"/>'
        )

    # nodes rectangles
    for n in sorted(nodes, key=lambda n: (x_level.get(n,0), y_pos.get(n,0))):
        x, y = x_level[n], y_pos[n]
        parts.append(
            f'<rect x="{x}" y="{y}" rx="10" ry="10" width="{node_w}" height="{node_h}" '
            f'fill="{zone_color(n)}" stroke="#333" stroke-width="1.2"/>'
        )

    # labels
    for n in sorted(nodes, key=lambda n: (x_level.get(n,0), y_pos.get(n,0))):
        x, y = x_level[n], y_pos[n]
        nid = node_ids.get(n, "")
        parts.append(f'<text x="{x+12}" y="{y+28}" font-size="14" fill="#111" font-weight="700">{svg_escape(nid)}</text>')
        lines = textwrap.wrap(svg_escape(n), width=34)
        for j, line in enumerate(lines[:2]):
            parts.append(f'<text x="{x+12}" y="{y+50 + j*16}" font-size="12" fill="#111">{line}</text>')

    parts.append("</svg></div></div></div>")
    return "\n".join(parts)

# -----------------------------
# Objetivos:
#  A) todos los tabla_destino (upstream)
#  B) todas las tablas crudas s_* que aparezcan como tabla_origen (downstream)
# -----------------------------
dest_targets = sorted(insumo_base["tabla_destino"].unique(), key=lambda s: s.lower())
raw_sources = sorted({t for t in insumo_base["tabla_origen"].unique() if parse_schema(t).startswith("s_")}, key=lambda s: s.lower())

objectives = [("upstream", t) for t in dest_targets] + [("downstream", s) for s in raw_sources]

# -----------------------------
# Generar tb_linaje y tb_html
# -----------------------------
tb_linaje_rows = []
tb_html_rows = []

for kind, obj in objectives:
    if kind == "upstream":
        nodes, edges = upstream_closure(obj)
        level_map = levels_upstream(edges, obj)
    else:
        nodes, edges = downstream_closure(obj)
        level_map = levels_downstream(edges, obj)

    node_ids = assign_ids_per_objective(nodes, level_map, kind)
    html = build_svg_html(nodes, edges, level_map, kind, node_ids)

    # tb_linaje
    for s, d, qry in edges:
        tb_linaje_rows.append({
            "tabla_linaje_objetivo": obj,
            "id": node_ids.get(d, ""),
            "tabla_origen": s,
            "tabla_destino": d,
            "query": qry
        })

    tb_html_rows.append({"tabla_destino": obj, "html": html})

tb_linaje_v2 = pd.DataFrame(tb_linaje_rows)
tb_html_v2 = pd.DataFrame(tb_html_rows)

# -----------------------------
# Guardar CSVs
# -----------------------------
out_dir = "./resultados_linaje"
linaje_path = out_dir + "/tb_linaje_generado.csv"
html_path = out_dir +"/tb_html_generado.csv"

tb_linaje_v2.to_csv(linaje_path, index=False, quoting=csv.QUOTE_ALL)
tb_html_v2.to_csv(html_path, index=False, quoting=csv.QUOTE_ALL)

summary = pd.DataFrame({
    "objetivos_destino_upstream": [len(dest_targets)],
    "objetivos_crudos_downstream": [len(raw_sources)],
    "total_objetivos": [len(objectives)],
    "filas_tb_linaje": [len(tb_linaje_v2)],
    "filas_tb_html": [len(tb_html_v2)]
})

(str(linaje_path), str(html_path), summary)


('./resultados_linaje/tb_linaje_generado.csv',
 './resultados_linaje/tb_html_generado.csv',
    objetivos_destino_upstream  objetivos_crudos_downstream  total_objetivos  \
 0                          12                            4               16   
 
    filas_tb_linaje  filas_tb_html  
 0               56             16  )

In [15]:
import pandas as pd, csv, textwrap
from pathlib import Path
from collections import defaultdict, deque

# Example insumo_base
insumo_rows = [
    ("s_core.clientes", "proceso_bipa_vpr.stg_clientes",
     "create table proceso_bipa_vpr.stg_clientes as select * from s_core.clientes;"),
    ("s_core.prestamos", "proceso_bipa_vpr.stg_prestamos",
     "create table proceso_bipa_vpr.stg_prestamos as select * from s_core.prestamos;"),
    ("proceso_bipa_vpr.stg_clientes", "proceso_bipa_vpr.dim_clientes",
     "create table proceso_bipa_vpr.dim_clientes as select ... from proceso_bipa_vpr.stg_clientes;"),
    ("proceso_bipa_vpr.stg_prestamos", "proceso_bipa_vpr.fact_prestamos_stg",
     "create table proceso_bipa_vpr.fact_prestamos_stg as select ... from proceso_bipa_vpr.stg_prestamos;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.fact_prestamos_stg", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),

    ("s_core.pagos", "proceso_bipa_vpr.stg_pagos",
     "create table proceso_bipa_vpr.stg_pagos as select * from s_core.pagos;"),
    ("proceso_bipa_vpr.stg_pagos", "proceso_bipa_vpr.fact_pagos_stg",
     "create table proceso_bipa_vpr.fact_pagos_stg as select ... from proceso_bipa_vpr.stg_pagos;"),
    ("proceso_bipa_vpr.fact_pagos_stg", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_stg;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),

    ("resultados_bipa_vpr.tb_fact_prestamos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_prestamos;"),
    ("resultados_bipa_vpr.tb_fact_pagos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_pagos;"),

    ("s_core.cartera", "proceso_riesgo.stg_cartera",
     "create table proceso_riesgo.stg_cartera as select * from s_core.cartera;"),
    ("proceso_riesgo.stg_cartera", "proceso_riesgo.mora_clientes",
     "create table proceso_riesgo.mora_clientes as select ... from proceso_riesgo.stg_cartera;"),
    ("proceso_riesgo.mora_clientes", "resultados_riesgo.tb_alertas_mora",
     "create table resultados_riesgo.tb_alertas_mora as select ... from proceso_riesgo.mora_clientes;"),
]
insumo_base = pd.DataFrame(insumo_rows, columns=["tabla_origen","tabla_destino","query"])

def parse_schema(fqn: str):
    fqn = (fqn or "").strip().lower()
    return fqn.split(".", 1)[0] if "." in fqn else fqn

def zone_prefix(fqn: str):
    schema = parse_schema(fqn)
    if schema.startswith("s_"):
        return "S"
    if schema.startswith("proceso"):
        return "P"
    if schema.startswith("resultados") or schema.startswith("resultado"):
        return "R"
    return "U"

def zone_color(fqn: str):
    z = zone_prefix(fqn)
    if z == "S":
        return "#e74c3c"
    if z == "P":
        return "#f1c40f"
    if z == "R":
        return "#2ecc71"
    return "#95a5a6"

def svg_escape(s: str):
    return (str(s)
            .replace("&","&amp;")
            .replace("<","&lt;")
            .replace(">","&gt;")
            .replace('"',"&quot;"))

incoming = defaultdict(list)
outgoing = defaultdict(list)
for _, r in insumo_base.iterrows():
    s = r["tabla_origen"].strip()
    d = r["tabla_destino"].strip()
    incoming[d].append((s, r["query"]))
    outgoing[s].append((d, r["query"]))

def upstream_closure(target: str):
    target = target.strip()
    visited = set([target])
    q = deque([target])
    edges = []
    while q:
        d = q.popleft()
        for s, qry in incoming.get(d, []):
            edges.append((s, d, qry))
            if s not in visited:
                visited.add(s)
                q.append(s)
    return visited, edges

def downstream_closure(root: str):
    root = root.strip()
    visited = set([root])
    q = deque([root])
    edges = []
    while q:
        s = q.popleft()
        for d, qry in outgoing.get(s, []):
            edges.append((s, d, qry))
            if d not in visited:
                visited.add(d)
                q.append(d)
    return visited, edges

def levels_upstream(edges, target):
    level = {target: 0}
    for _ in range(200):
        changed = False
        for s, d, _ in edges:
            if d in level:
                cand = level[d] + 1
                if s not in level or cand > level[s]:
                    level[s] = cand
                    changed = True
        if not changed:
            break
    return level

def levels_downstream(edges, root):
    level = {root: 0}
    for _ in range(200):
        changed = False
        for s, d, _ in edges:
            if s in level:
                cand = level[s] + 1
                if d not in level or cand > level[d]:
                    level[d] = cand
                    changed = True
        if not changed:
            break
    return level

def assign_ids_per_objective(nodes, level_map, kind):
    def sort_key(n):
        lv = level_map.get(n, -1)
        return ((-lv, n.lower()) if kind == "upstream" else (lv, n.lower()))
    ordered = sorted(nodes, key=sort_key)

    counters = {"S":1, "P":1, "R":1, "U":1}
    node_id = {}
    for n in ordered:
        z = zone_prefix(n)
        node_id[n] = f"{z}{counters[z]}"
        counters[z] += 1
    return node_id

def build_svg_html_v3(nodes, edges, level_map, kind, node_ids):
    dx = 420
    dy = 140
    node_w, node_h = 270, 74

    max_level = max(level_map.values()) if level_map else 0

    if kind == "upstream":
        x_level = {n: (max_level - level_map.get(n, max_level)) * dx for n in nodes}
    else:
        x_level = {n: level_map.get(n, 0) * dx for n in nodes}

    buckets = defaultdict(list)
    for n in nodes:
        buckets[level_map.get(n, 0)].append(n)
    for k in buckets:
        buckets[k].sort(key=lambda n: n.lower())

    y_pos = {}
    for k, ns in buckets.items():
        for i, n in enumerate(ns):
            y_pos[n] = 60 + i * dy

    width = (max_level + 1) * dx + 520
    height = (max(y_pos.values()) if y_pos else 0) + 260

    parts = []
    parts.append('<div style="width:100%;height:100%;min-height:650px;overflow:scroll;background:#ffffff;">')
    parts.append(f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}" style="display:block;background:#ffffff;">')

    parts.append("""
<defs>
  <marker id="arrow" viewBox="0 0 10 10" refX="10" refY="5" markerWidth="7" markerHeight="7" orient="auto-start-reverse">
    <path d="M 0 0 L 10 5 L 0 10 z" fill="#555"></path>
  </marker>
</defs>
""".strip())

    for s, d, _ in edges:
        if s not in x_level or d not in x_level:
            continue
        x1 = x_level[s] + node_w
        y1 = y_pos[s] + node_h/2
        x2 = x_level[d]
        y2 = y_pos[d] + node_h/2
        mx = (x1 + x2) / 2
        parts.append(
            f'<path d="M {x1} {y1} C {mx} {y1}, {mx} {y2}, {x2} {y2}" '
            f'stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrow)"/>'
        )

    for n in sorted(nodes, key=lambda n: (x_level.get(n,0), y_pos.get(n,0))):
        x, y = x_level[n], y_pos[n]
        parts.append(
            f'<rect x="{x}" y="{y}" rx="10" ry="10" width="{node_w}" height="{node_h}" '
            f'fill="{zone_color(n)}" stroke="#333" stroke-width="1.2"/>'
        )

    for n in sorted(nodes, key=lambda n: (x_level.get(n,0), y_pos.get(n,0))):
        x, y = x_level[n], y_pos[n]
        nid = node_ids.get(n, "")
        parts.append(f'<text x="{x+12}" y="{y+28}" font-size="14" fill="#111" font-weight="700">{svg_escape(nid)}</text>')
        lines = textwrap.wrap(svg_escape(n), width=34)
        for j, line in enumerate(lines[:2]):
            parts.append(f'<text x="{x+12}" y="{y+50 + j*16}" font-size="12" fill="#111">{line}</text>')

    parts.append("</svg></div>")
    return "\n".join(parts)

dest_targets = sorted(insumo_base["tabla_destino"].unique(), key=lambda s: s.lower())
raw_sources = sorted({t for t in insumo_base["tabla_origen"].unique() if parse_schema(t).startswith("s_")}, key=lambda s: s.lower())
objectives = [("upstream", t) for t in dest_targets] + [("downstream", s) for s in raw_sources]

tb_linaje_rows = []
tb_html_rows = []

for kind, obj in objectives:
    if kind == "upstream":
        nodes, edges = upstream_closure(obj)
        level_map = levels_upstream(edges, obj)
    else:
        nodes, edges = downstream_closure(obj)
        level_map = levels_downstream(edges, obj)

    node_ids = assign_ids_per_objective(nodes, level_map, kind)
    html = build_svg_html_v3(nodes, edges, level_map, kind, node_ids)

    for s, d, qry in edges:
        tb_linaje_rows.append({
            "tabla_linaje_objetivo": obj,
            "id": node_ids.get(d, ""),
            "tabla_origen": s,
            "tabla_destino": d,
            "query": qry
        })

    tb_html_rows.append({"tabla_destino": obj, "html": html})

tb_linaje_v3 = pd.DataFrame(tb_linaje_rows)
tb_html_v3 = pd.DataFrame(tb_html_rows)

out_dir = "./resultados_linaje"
linaje_path = out_dir + "/tb_linaje_generado.csv"
html_path = out_dir +"/tb_html_generado.csv"

tb_linaje_v3.to_csv(linaje_path, index=False, quoting=csv.QUOTE_ALL)
tb_html_v3.to_csv(html_path, index=False, quoting=csv.QUOTE_ALL)

(str(linaje_path), str(html_path), tb_html_v3.shape, tb_linaje_v3.shape)


('./resultados_linaje/tb_linaje_generado.csv',
 './resultados_linaje/tb_html_generado.csv',
 (16, 2),
 (56, 5))

test

In [4]:
insumo_rows = [
    # =========================
    # Fuentes crudas -> staging (20)
    # =========================
    ("s_core.clientes", "proceso_bipa_vpr.stg_clientes",
     "create table proceso_bipa_vpr.stg_clientes as select * from s_core.clientes;"),
    ("s_core.prestamos", "proceso_bipa_vpr.stg_prestamos",
     "create table proceso_bipa_vpr.stg_prestamos as select * from s_core.prestamos;"),
    ("s_core.pagos", "proceso_bipa_vpr.stg_pagos",
     "create table proceso_bipa_vpr.stg_pagos as select * from s_core.pagos;"),
    ("s_core.cartera", "proceso_riesgo.stg_cartera",
     "create table proceso_riesgo.stg_cartera as select * from s_core.cartera;"),
    ("s_core.sucursales", "proceso_bipa_vpr.stg_sucursales",
     "create table proceso_bipa_vpr.stg_sucursales as select * from s_core.sucursales;"),
    ("s_core.productos", "proceso_bipa_vpr.stg_productos",
     "create table proceso_bipa_vpr.stg_productos as select * from s_core.productos;"),
    ("s_core.tasas", "proceso_bipa_vpr.stg_tasas",
     "create table proceso_bipa_vpr.stg_tasas as select * from s_core.tasas;"),
    ("s_core.garantias", "proceso_bipa_vpr.stg_garantias",
     "create table proceso_bipa_vpr.stg_garantias as select * from s_core.garantias;"),
    ("s_core.transacciones", "proceso_bipa_vpr.stg_transacciones",
     "create table proceso_bipa_vpr.stg_transacciones as select * from s_core.transacciones;"),
    ("s_core.calendario", "proceso_bipa_vpr.stg_calendario",
     "create table proceso_bipa_vpr.stg_calendario as select * from s_core.calendario;"),
    ("s_core.oficiales", "proceso_bipa_vpr.stg_oficiales",
     "create table proceso_bipa_vpr.stg_oficiales as select * from s_core.oficiales;"),
    ("s_core.segmentos", "proceso_bipa_vpr.stg_segmentos",
     "create table proceso_bipa_vpr.stg_segmentos as select * from s_core.segmentos;"),
    ("s_core.monedas", "proceso_bipa_vpr.stg_monedas",
     "create table proceso_bipa_vpr.stg_monedas as select * from s_core.monedas;"),
    ("s_core.tipo_cambio", "proceso_bipa_vpr.stg_tipo_cambio",
     "create table proceso_bipa_vpr.stg_tipo_cambio as select * from s_core.tipo_cambio;"),
    ("s_core.cobros", "proceso_bipa_vpr.stg_cobros",
     "create table proceso_bipa_vpr.stg_cobros as select * from s_core.cobros;"),
    ("s_core.mora_hist", "proceso_riesgo.stg_mora_hist",
     "create table proceso_riesgo.stg_mora_hist as select * from s_core.mora_hist;"),
    ("s_core.score_externo", "proceso_riesgo.stg_score_externo",
     "create table proceso_riesgo.stg_score_externo as select * from s_core.score_externo;"),
    ("s_core.ingresos", "proceso_bipa_vpr.stg_ingresos",
     "create table proceso_bipa_vpr.stg_ingresos as select * from s_core.ingresos;"),
    ("s_core.egresos", "proceso_bipa_vpr.stg_egresos",
     "create table proceso_bipa_vpr.stg_egresos as select * from s_core.egresos;"),
    ("s_core.localizacion", "proceso_bipa_vpr.stg_localizacion",
     "create table proceso_bipa_vpr.stg_localizacion as select * from s_core.localizacion;"),

    # =========================
    # Staging -> tablas proceso base
    # =========================
    ("proceso_bipa_vpr.stg_clientes", "proceso_bipa_vpr.dim_clientes",
     "create table proceso_bipa_vpr.dim_clientes as select ... from proceso_bipa_vpr.stg_clientes;"),
    ("proceso_bipa_vpr.stg_segmentos", "proceso_bipa_vpr.dim_clientes",
     "create table proceso_bipa_vpr.dim_clientes as select ... from proceso_bipa_vpr.stg_clientes c left join proceso_bipa_vpr.stg_segmentos s on ...;"),
    ("proceso_bipa_vpr.stg_localizacion", "proceso_bipa_vpr.dim_clientes",
     "create table proceso_bipa_vpr.dim_clientes as select ... from proceso_bipa_vpr.stg_clientes c left join proceso_bipa_vpr.stg_localizacion l on ...;"),

    ("proceso_bipa_vpr.stg_productos", "proceso_bipa_vpr.dim_productos",
     "create table proceso_bipa_vpr.dim_productos as select ... from proceso_bipa_vpr.stg_productos;"),
    ("proceso_bipa_vpr.stg_tasas", "proceso_bipa_vpr.dim_productos",
     "create table proceso_bipa_vpr.dim_productos as select ... from proceso_bipa_vpr.stg_productos p left join proceso_bipa_vpr.stg_tasas t on ...;"),

    ("proceso_bipa_vpr.stg_sucursales", "proceso_bipa_vpr.dim_sucursales",
     "create table proceso_bipa_vpr.dim_sucursales as select ... from proceso_bipa_vpr.stg_sucursales;"),
    ("proceso_bipa_vpr.stg_oficiales", "proceso_bipa_vpr.dim_oficiales",
     "create table proceso_bipa_vpr.dim_oficiales as select ... from proceso_bipa_vpr.stg_oficiales;"),
    ("proceso_bipa_vpr.stg_calendario", "proceso_bipa_vpr.dim_tiempo",
     "create table proceso_bipa_vpr.dim_tiempo as select ... from proceso_bipa_vpr.stg_calendario;"),
    ("proceso_bipa_vpr.stg_monedas", "proceso_bipa_vpr.dim_monedas",
     "create table proceso_bipa_vpr.dim_monedas as select ... from proceso_bipa_vpr.stg_monedas;"),
    ("proceso_bipa_vpr.stg_tipo_cambio", "proceso_bipa_vpr.dim_tipo_cambio",
     "create table proceso_bipa_vpr.dim_tipo_cambio as select ... from proceso_bipa_vpr.stg_tipo_cambio;"),

    ("proceso_bipa_vpr.stg_prestamos", "proceso_bipa_vpr.fact_prestamos_stg",
     "create table proceso_bipa_vpr.fact_prestamos_stg as select ... from proceso_bipa_vpr.stg_prestamos;"),
    ("proceso_bipa_vpr.stg_garantias", "proceso_bipa_vpr.fact_prestamos_stg",
     "create table proceso_bipa_vpr.fact_prestamos_stg as select ... from proceso_bipa_vpr.stg_prestamos p left join proceso_bipa_vpr.stg_garantias g on ...;"),
    ("proceso_bipa_vpr.stg_tasas", "proceso_bipa_vpr.fact_prestamos_stg",
     "create table proceso_bipa_vpr.fact_prestamos_stg as select ... from proceso_bipa_vpr.stg_prestamos p left join proceso_bipa_vpr.stg_tasas t on ...;"),

    ("proceso_bipa_vpr.stg_pagos", "proceso_bipa_vpr.fact_pagos_stg",
     "create table proceso_bipa_vpr.fact_pagos_stg as select ... from proceso_bipa_vpr.stg_pagos;"),
    ("proceso_bipa_vpr.stg_cobros", "proceso_bipa_vpr.fact_pagos_stg",
     "create table proceso_bipa_vpr.fact_pagos_stg as select ... from proceso_bipa_vpr.stg_pagos p left join proceso_bipa_vpr.stg_cobros c on ...;"),

    ("proceso_bipa_vpr.stg_transacciones", "proceso_bipa_vpr.fact_transacciones_stg",
     "create table proceso_bipa_vpr.fact_transacciones_stg as select ... from proceso_bipa_vpr.stg_transacciones;"),
    ("proceso_bipa_vpr.stg_ingresos", "proceso_bipa_vpr.fact_ingresos_stg",
     "create table proceso_bipa_vpr.fact_ingresos_stg as select ... from proceso_bipa_vpr.stg_ingresos;"),
    ("proceso_bipa_vpr.stg_egresos", "proceso_bipa_vpr.fact_egresos_stg",
     "create table proceso_bipa_vpr.fact_egresos_stg as select ... from proceso_bipa_vpr.stg_egresos;"),

    ("proceso_riesgo.stg_cartera", "proceso_riesgo.mora_clientes",
     "create table proceso_riesgo.mora_clientes as select ... from proceso_riesgo.stg_cartera;"),
    ("proceso_riesgo.stg_mora_hist", "proceso_riesgo.mora_clientes",
     "create table proceso_riesgo.mora_clientes as select ... from proceso_riesgo.stg_cartera c left join proceso_riesgo.stg_mora_hist m on ...;"),

    ("proceso_riesgo.stg_score_externo", "proceso_riesgo.score_enriquecido",
     "create table proceso_riesgo.score_enriquecido as select ... from proceso_riesgo.stg_score_externo;"),
    ("proceso_bipa_vpr.stg_clientes", "proceso_riesgo.score_enriquecido",
     "create table proceso_riesgo.score_enriquecido as select ... from proceso_riesgo.stg_score_externo s left join proceso_bipa_vpr.stg_clientes c on ...;"),

    # =========================
    # Proceso -> proceso intermedio (más profundidad)
    # =========================
    ("proceso_bipa_vpr.fact_prestamos_stg", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg;"),
    ("proceso_bipa_vpr.dim_clientes", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_productos", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_productos dp on ...;"),
    ("proceso_bipa_vpr.dim_sucursales", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_sucursales ds on ...;"),
    ("proceso_bipa_vpr.dim_oficiales", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_oficiales dof on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_tiempo dt on ...;"),

    ("proceso_bipa_vpr.fact_pagos_stg", "proceso_bipa_vpr.fact_pagos_enriquecido",
     "create table proceso_bipa_vpr.fact_pagos_enriquecido as select ... from proceso_bipa_vpr.fact_pagos_stg;"),
    ("proceso_bipa_vpr.dim_clientes", "proceso_bipa_vpr.fact_pagos_enriquecido",
     "create table proceso_bipa_vpr.fact_pagos_enriquecido as select ... from proceso_bipa_vpr.fact_pagos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "proceso_bipa_vpr.fact_pagos_enriquecido",
     "create table proceso_bipa_vpr.fact_pagos_enriquecido as select ... from proceso_bipa_vpr.fact_pagos_stg fp join proceso_bipa_vpr.dim_tiempo dt on ...;"),
    ("proceso_bipa_vpr.dim_monedas", "proceso_bipa_vpr.fact_pagos_enriquecido",
     "create table proceso_bipa_vpr.fact_pagos_enriquecido as select ... from proceso_bipa_vpr.fact_pagos_stg fp join proceso_bipa_vpr.dim_monedas dm on ...;"),

    ("proceso_bipa_vpr.fact_transacciones_stg", "proceso_bipa_vpr.fact_transacciones_enriquecido",
     "create table proceso_bipa_vpr.fact_transacciones_enriquecido as select ... from proceso_bipa_vpr.fact_transacciones_stg;"),
    ("proceso_bipa_vpr.dim_clientes", "proceso_bipa_vpr.fact_transacciones_enriquecido",
     "create table proceso_bipa_vpr.fact_transacciones_enriquecido as select ... from proceso_bipa_vpr.fact_transacciones_stg ft join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "proceso_bipa_vpr.fact_transacciones_enriquecido",
     "create table proceso_bipa_vpr.fact_transacciones_enriquecido as select ... from proceso_bipa_vpr.fact_transacciones_stg ft join proceso_bipa_vpr.dim_tiempo dt on ...;"),
    ("proceso_bipa_vpr.dim_sucursales", "proceso_bipa_vpr.fact_transacciones_enriquecido",
     "create table proceso_bipa_vpr.fact_transacciones_enriquecido as select ... from proceso_bipa_vpr.fact_transacciones_stg ft join proceso_bipa_vpr.dim_sucursales ds on ...;"),

    ("proceso_riesgo.mora_clientes", "proceso_riesgo.mora_enriquecida",
     "create table proceso_riesgo.mora_enriquecida as select ... from proceso_riesgo.mora_clientes;"),
    ("proceso_riesgo.score_enriquecido", "proceso_riesgo.mora_enriquecida",
     "create table proceso_riesgo.mora_enriquecida as select ... from proceso_riesgo.mora_clientes m left join proceso_riesgo.score_enriquecido s on ...;"),
    ("proceso_bipa_vpr.dim_clientes", "proceso_riesgo.mora_enriquecida",
     "create table proceso_riesgo.mora_enriquecida as select ... from proceso_riesgo.mora_clientes m left join proceso_bipa_vpr.dim_clientes dc on ...;"),

    # =========================
    # Proceso -> resultados (facts y alertas)
    # =========================
    ("proceso_bipa_vpr.fact_prestamos_enriquecido", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_enriquecido;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_enriquecido fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_productos", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_enriquecido fp join proceso_bipa_vpr.dim_productos dp on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_enriquecido fp join proceso_bipa_vpr.dim_tiempo dt on ...;"),

    ("proceso_bipa_vpr.fact_pagos_enriquecido", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_enriquecido;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_enriquecido fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_enriquecido fp join proceso_bipa_vpr.dim_tiempo dt on ...;"),

    ("proceso_bipa_vpr.fact_transacciones_enriquecido", "resultados_bipa_vpr.tb_fact_transacciones",
     "create table resultados_bipa_vpr.tb_fact_transacciones as select ... from proceso_bipa_vpr.fact_transacciones_enriquecido;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_transacciones",
     "create table resultados_bipa_vpr.tb_fact_transacciones as select ... from proceso_bipa_vpr.fact_transacciones_enriquecido ft join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "resultados_bipa_vpr.tb_fact_transacciones",
     "create table resultados_bipa_vpr.tb_fact_transacciones as select ... from proceso_bipa_vpr.fact_transacciones_enriquecido ft join proceso_bipa_vpr.dim_tiempo dt on ...;"),

    ("proceso_riesgo.mora_enriquecida", "resultados_riesgo.tb_alertas_mora",
     "create table resultados_riesgo.tb_alertas_mora as select ... from proceso_riesgo.mora_enriquecida;"),
    ("proceso_riesgo.score_enriquecido", "resultados_riesgo.tb_alertas_mora",
     "create table resultados_riesgo.tb_alertas_mora as select ... from proceso_riesgo.mora_enriquecida m left join proceso_riesgo.score_enriquecido s on ...;"),

    ("proceso_riesgo.mora_enriquecida", "resultados_riesgo.tb_score_riesgo",
     "create table resultados_riesgo.tb_score_riesgo as select ... from proceso_riesgo.mora_enriquecida;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_riesgo.tb_score_riesgo",
     "create table resultados_riesgo.tb_score_riesgo as select ... from proceso_riesgo.mora_enriquecida m join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "resultados_riesgo.tb_score_riesgo",
     "create table resultados_riesgo.tb_score_riesgo as select ... from proceso_riesgo.mora_enriquecida m join proceso_bipa_vpr.dim_tiempo dt on ...;"),

    # =========================
    # Resultados -> resultados (reportes)
    # =========================
    ("resultados_bipa_vpr.tb_fact_prestamos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_prestamos;"),
    ("resultados_bipa_vpr.tb_fact_pagos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_pagos;"),
    ("resultados_bipa_vpr.tb_fact_transacciones", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_transacciones;"),
    ("resultados_riesgo.tb_alertas_mora", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_riesgo.tb_alertas_mora;"),

    ("resultados_bipa_vpr.tb_fact_prestamos", "resultados_bipa_vpr.tb_reporte_cartera",
     "create table resultados_bipa_vpr.tb_reporte_cartera as select ... from resultados_bipa_vpr.tb_fact_prestamos;"),
    ("resultados_riesgo.tb_score_riesgo", "resultados_bipa_vpr.tb_reporte_cartera",
     "create table resultados_bipa_vpr.tb_reporte_cartera as select ... from resultados_riesgo.tb_score_riesgo;"),

    ("resultados_bipa_vpr.tb_reporte_mensual", "resultados_bipa_vpr.tb_dashboard_ejecutivo",
     "create table resultados_bipa_vpr.tb_dashboard_ejecutivo as select ... from resultados_bipa_vpr.tb_reporte_mensual;"),
    ("resultados_bipa_vpr.tb_reporte_cartera", "resultados_bipa_vpr.tb_dashboard_ejecutivo",
     "create table resultados_bipa_vpr.tb_dashboard_ejecutivo as select ... from resultados_bipa_vpr.tb_reporte_cartera;"),
    ("resultados_riesgo.tb_alertas_mora", "resultados_bipa_vpr.tb_dashboard_ejecutivo",
     "create table resultados_bipa_vpr.tb_dashboard_ejecutivo as select ... from resultados_riesgo.tb_alertas_mora;"),
    ("resultados_riesgo.tb_score_riesgo", "resultados_bipa_vpr.tb_dashboard_ejecutivo",
     "create table resultados_bipa_vpr.tb_dashboard_ejecutivo as select ... from resultados_riesgo.tb_score_riesgo;"),

    # =========================
    # Relaciones extra (complejidad)
    # =========================
    ("proceso_bipa_vpr.stg_clientes", "proceso_riesgo.score_enriquecido",
     "insert overwrite table proceso_riesgo.score_enriquecido select ... from proceso_riesgo.stg_score_externo se join proceso_bipa_vpr.stg_clientes c on ...;"),
    ("proceso_bipa_vpr.fact_pagos_stg", "proceso_riesgo.mora_enriquecida",
     "insert overwrite table proceso_riesgo.mora_enriquecida select ... from proceso_riesgo.mora_clientes m left join proceso_bipa_vpr.fact_pagos_stg p on ...;"),
    ("proceso_bipa_vpr.fact_prestamos_enriquecido", "resultados_riesgo.tb_score_riesgo",
     "insert into resultados_riesgo.tb_score_riesgo select ... from proceso_bipa_vpr.fact_prestamos_enriquecido fp;"),
    ("resultados_bipa_vpr.tb_fact_pagos", "resultados_bipa_vpr.tb_reporte_cartera",
     "insert into resultados_bipa_vpr.tb_reporte_cartera select ... from resultados_bipa_vpr.tb_fact_pagos;"),
    ("resultados_bipa_vpr.tb_fact_transacciones", "resultados_bipa_vpr.tb_reporte_cartera",
     "insert into resultados_bipa_vpr.tb_reporte_cartera select ... from resultados_bipa_vpr.tb_fact_transacciones;"),

    # =========================
    # Self-loops (para probar lógica de visitados/duplicados)
    # =========================
    ("proceso_bipa_vpr.fact_pagos_stg", "proceso_bipa_vpr.fact_pagos_stg",
     "insert overwrite table proceso_bipa_vpr.fact_pagos_stg select ... from proceso_bipa_vpr.fact_pagos_stg;"),
    ("resultados_bipa_vpr.tb_reporte_mensual", "resultados_bipa_vpr.tb_reporte_mensual",
     "insert overwrite table resultados_bipa_vpr.tb_reporte_mensual select ... from resultados_bipa_vpr.tb_reporte_mensual;"),
]

insumo_base = pd.DataFrame(insumo_rows, columns=["tabla_origen","tabla_destino","query"])

In [6]:
import pandas as pd, csv, textwrap
from pathlib import Path
import json
from collections import defaultdict, deque

# -----------------------------
# Ejemplo de insumo_base (igual que antes)
# -----------------------------
""" 
insumo_rows = [
    ("s_core.clientes", "proceso_bipa_vpr.stg_clientes",
     "create table proceso_bipa_vpr.stg_clientes as select * from s_core.clientes;"),
    ("s_core.prestamos", "proceso_bipa_vpr.stg_prestamos",
     "create table proceso_bipa_vpr.stg_prestamos as select * from s_core.prestamos;"),
    ("proceso_bipa_vpr.stg_clientes", "proceso_bipa_vpr.dim_clientes",
     "create table proceso_bipa_vpr.dim_clientes as select ... from proceso_bipa_vpr.stg_clientes;"),
    ("proceso_bipa_vpr.stg_prestamos", "proceso_bipa_vpr.fact_prestamos_stg",
     "create table proceso_bipa_vpr.fact_prestamos_stg as select ... from proceso_bipa_vpr.stg_prestamos;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.fact_prestamos_stg", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),

    ("s_core.pagos", "proceso_bipa_vpr.stg_pagos",
     "create table proceso_bipa_vpr.stg_pagos as select * from s_core.pagos;"),
    ("proceso_bipa_vpr.stg_pagos", "proceso_bipa_vpr.fact_pagos_stg",
     "create table proceso_bipa_vpr.fact_pagos_stg as select ... from proceso_bipa_vpr.stg_pagos;"),
    ("proceso_bipa_vpr.fact_pagos_stg", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_stg;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),

    ("resultados_bipa_vpr.tb_fact_prestamos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_prestamos;"),
    ("resultados_bipa_vpr.tb_fact_pagos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_pagos;"),

    ("s_core.cartera", "proceso_riesgo.stg_cartera",
     "create table proceso_riesgo.stg_cartera as select * from s_core.cartera;"),
    ("proceso_riesgo.stg_cartera", "proceso_riesgo.mora_clientes",
     "create table proceso_riesgo.mora_clientes as select ... from proceso_riesgo.stg_cartera;"),
    ("proceso_riesgo.mora_clientes", "resultados_riesgo.tb_alertas_mora",
     "create table resultados_riesgo.tb_alertas_mora as select ... from proceso_riesgo.mora_clientes;"),
]
insumo_base = pd.DataFrame(insumo_rows, columns=["tabla_origen","tabla_destino","query"])

 """
# -----------------------------
# Utilidades
# -----------------------------
def parse_schema(fqn: str):
    fqn = (fqn or "").strip().lower()
    return fqn.split(".", 1)[0] if "." in fqn else fqn

def zone_prefix(fqn: str):
    schema = parse_schema(fqn)
    if schema.startswith("s_"):
        return "S"
    if schema.startswith("proceso"):
        return "P"
    if schema.startswith("resultados") or schema.startswith("resultado"):
        return "R"
    return "U"

def zone_color(fqn: str):
    z = zone_prefix(fqn)
    if z == "S":
        return "#e74c3c"
    if z == "P":
        return "#f1c40f"
    if z == "R":
        return "#2ecc71"
    return "#95a5a6"

def svg_escape(s: str):
    return (str(s)
            .replace("&","&amp;")
            .replace("<","&lt;")
            .replace(">","&gt;")
            .replace('"',"&quot;"))

# -----------------------------
# Construir índices
# -----------------------------
incoming = defaultdict(list)  # dst -> [(src, query)]
outgoing = defaultdict(list)  # src -> [(dst, query)]
for _, r in insumo_base.iterrows():
    s = r["tabla_origen"].strip()
    d = r["tabla_destino"].strip()
    incoming[d].append((s, r["query"]))
    outgoing[s].append((d, r["query"]))

In [ ]:

# -----------------------------
# Closures
# -----------------------------
def upstream_closure(target: str):
    """
    Linaje upstream SIN duplicados.
    - visited_nodes evita re-procesar nodos (ciclos)
    - seen_edges evita repetir relaciones
    - si hay self-loop (A->A) se incluye 1 vez
    """
    target = target.strip()
    visited_nodes = set([target])
    q = deque([target])

    seen_edges = set()   # (src, dst, query)
    edges = []

    while q:
        d = q.popleft()

        for s, qry in incoming.get(d, []):
            s = (s or "").strip()
            edge_key = (s, d, (qry or "").strip())

            if edge_key not in seen_edges:
                seen_edges.add(edge_key)
                edges.append((s, d, qry))

            # Encolar sólo si el nodo no fue visitado (evita ciclos)
            if s not in visited_nodes:
                visited_nodes.add(s)
                q.append(s)

    return visited_nodes, edges


def downstream_closure(root: str):
    """
    Linaje downstream SIN duplicados.
    - visited_nodes evita re-procesar nodos (ciclos)
    - seen_edges evita repetir relaciones
    - si hay self-loop (A->A) se incluye 1 vez
    """
    root = root.strip()
    visited_nodes = set([root])
    q = deque([root])

    seen_edges = set()   # (src, dst, query)
    edges = []

    while q:
        s = q.popleft()

        for d, qry in outgoing.get(s, []):
            d = (d or "").strip()
            edge_key = (s, d, (qry or "").strip())

            if edge_key not in seen_edges:
                seen_edges.add(edge_key)
                edges.append((s, d, qry))

            if d not in visited_nodes:
                visited_nodes.add(d)
                q.append(d)

    return visited_nodes, edges

# -----------------------------
# Levels para layout
# -----------------------------
def levels_upstream(edges, target):
    level = {target: 0}
    for _ in range(200):
        changed = False
        for s, d, _ in edges:
            if d in level:
                cand = level[d] + 1
                if s not in level or cand > level[s]:
                    level[s] = cand
                    changed = True
        if not changed:
            break
    return level

def levels_downstream(edges, root):
    level = {root: 0}
    for _ in range(200):
        changed = False
        for s, d, _ in edges:
            if s in level:
                cand = level[s] + 1
                if d not in level or cand > level[d]:
                    level[d] = cand
                    changed = True
        if not changed:
            break
    return level


In [ ]:


# -----------------------------
# IDs (reinicio por objetivo)
# -----------------------------
def assign_ids_per_objective(nodes, level_map, kind):
    def sort_key(n):
        lv = level_map.get(n, -1)
        return ((-lv, n.lower()) if kind == "upstream" else (lv, n.lower()))
    ordered = sorted(nodes, key=sort_key)

    counters = {"S":1, "P":1, "R":1, "U":1}
    node_id = {}
    for n in ordered:
        z = zone_prefix(n)
        node_id[n] = f"{z}{counters[z]}"
        counters[z] += 1
    return node_id

# -----------------------------
# SVG/HTML (con scrollbars "fijos" al borde del visual)
# -----------------------------
def build_svg_html(nodes, edges, level_map, kind, node_ids, uid="g1"):
    dx = 350
    dy = 140
    node_w, node_h = 270, 74

    max_level = max(level_map.values()) if level_map else 0

    if kind == "upstream":
        x_level = {n: (max_level - level_map.get(n, max_level)) * dx for n in nodes}
    else:
        x_level = {n: level_map.get(n, 0) * dx for n in nodes}

    buckets = defaultdict(list)
    for n in nodes:
        buckets[level_map.get(n, 0)].append(n)
    for k in buckets:
        buckets[k].sort(key=lambda n: n.lower())

    y_pos = {}
    for k, ns in buckets.items():
        for i, n in enumerate(ns):
            y_pos[n] = 60 + i * dy

    width = (max_level + 1) * dx + 520
    height = (max(y_pos.values()) if y_pos else 0) + 260

    parts = []

    # Contenedor general con toolbar + viewport
    parts.append(f"""
<div style="width:100%; height:100%; background:#ffffff; font-family:Segoe UI, Arial;">
  <div style="display:flex; gap:8px; align-items:center; padding:6px 8px; border-bottom:1px solid #ddd;">
    <button id="zin_{uid}" style="padding:4px 10px;">zoom in</button>
    <button id="zout_{uid}" style="padding:4px 10px;">zoom out</button>
    <button id="zreset_{uid}" style="padding:4px 10px;">Reset</button>
    <span id="zlbl_{uid}" style="margin-left:8px; color:#444; font-size:12px;">100%</span>
  </div>

  <div id="vp_{uid}" style="height:calc(320px - 38px); overflow:scroll; background:#fff;">
    <!-- content escalable -->
    <div id="content_{uid}" style="transform:scale(1); transform-origin:0 0; width:{width}px; height:{height}px;">
      <svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}" style="display:block; background:#ffffff;">
""".strip())

    # defs
    parts.append("""
<defs>
  <marker id="arrow" viewBox="0 0 10 10" refX="10" refY="5" markerWidth="7" markerHeight="7" orient="auto-start-reverse">
    <path d="M 0 0 L 10 5 L 0 10 z" fill="#555"></path>
  </marker>
</defs>
""".strip())

    # edges
    for s, d, _ in edges:
        if s not in x_level or d not in x_level:
            continue
        x1 = x_level[s] + node_w
        y1 = y_pos[s] + node_h / 2
        x2 = x_level[d]
        y2 = y_pos[d] + node_h / 2
        mx = (x1 + x2) / 2
        parts.append(
            f'<path d="M {x1} {y1} C {mx} {y1}, {mx} {y2}, {x2} {y2}" '
            f'stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrow)"/>'
        )

    # nodes
    for n in sorted(nodes, key=lambda n: (x_level.get(n, 0), y_pos.get(n, 0))):
        x, y = x_level[n], y_pos[n]
        parts.append(
            f'<rect x="{x}" y="{y}" rx="10" ry="10" width="{node_w}" height="{node_h}" '
            f'fill="{zone_color(n)}" stroke="#333" stroke-width="1.2"/>'
        )

    # labels
    for n in sorted(nodes, key=lambda n: (x_level.get(n, 0), y_pos.get(n, 0))):
        x, y = x_level[n], y_pos[n]
        nid = node_ids.get(n, "")
        parts.append(f'<text x="{x+12}" y="{y+28}" font-size="14" fill="#111" font-weight="700">{svg_escape(nid)}</text>')
        lines = textwrap.wrap(svg_escape(n), width=34)
        for j, line in enumerate(lines[:2]):
            parts.append(f'<text x="{x+12}" y="{y+50 + j*16}" font-size="12" fill="#111">{line}</text>')

    # Cierre svg + JS zoom
    parts.append(f"""
      </svg>
    </div>
  </div>
</div>

<script>
(function() {{
  var scale = 1.0;
  var minS = 0.4, maxS = 2.5, step = 0.1;

  var content = document.getElementById("content_{uid}");
  var lbl = document.getElementById("zlbl_{uid}");

  function apply() {{
    if(!content) return;
    content.style.transform = "scale(" + scale.toFixed(2) + ")";
    if(lbl) lbl.textContent = Math.round(scale * 100) + "%";
  }}

  var zin = document.getElementById("zin_{uid}");
  var zout = document.getElementById("zout_{uid}");
  var zreset = document.getElementById("zreset_{uid}");

  if(zin) zin.addEventListener("click", function() {{
    scale = Math.min(maxS, scale + step);
    apply();
  }});

  if(zout) zout.addEventListener("click", function() {{
    scale = Math.max(minS, scale - step);
    apply();
  }});

  if(zreset) zreset.addEventListener("click", function() {{
    scale = 1.0;
    apply();
  }});

  apply();
}})();
</script>
""".strip())

    return "\n".join(parts)

def build_svg_html(nodes, edges, level_map, kind, node_ids, uid="g1", viewport_h=320):
    dx = 350
    dy = 140
    node_w, node_h = 270, 74

    max_level = max(level_map.values()) if level_map else 0

    if kind == "upstream":
        x_level = {n: (max_level - level_map.get(n, max_level)) * dx for n in nodes}
    else:
        x_level = {n: level_map.get(n, 0) * dx for n in nodes}

    buckets = defaultdict(list)
    for n in nodes:
        buckets[level_map.get(n, 0)].append(n)
    for k in buckets:
        buckets[k].sort(key=lambda n: n.lower())

    y_pos = {}
    for k, ns in buckets.items():
        for i, n in enumerate(ns):
            y_pos[n] = 60 + i * dy

    base_w = (max_level + 1) * dx + 520
    base_h = (max(y_pos.values()) if y_pos else 0) + 260

    toolbar_h = 38  # alto aprox. de la barra de botones

    parts = []
    parts.append(f"""
<div style="width:100%; height:{viewport_h}px; overflow:hidden; background:#ffffff; font-family:Segoe UI, Arial;">
  <div style="display:flex; gap:8px; align-items:center; padding:6px 8px; height:{toolbar_h}px; box-sizing:border-box; border-bottom:1px solid #ddd;">
    <button id="zin_{uid}" style="padding:4px 10px;">zoom in</button>
    <button id="zout_{uid}" style="padding:4px 10px;">zoom out</button>
    <button id="zreset_{uid}" style="padding:4px 10px;">Reset</button>
    <span id="zlbl_{uid}" style="margin-left:8px; color:#444; font-size:12px;">100%</span>
    <span style="margin-left:10px; color:#777; font-size:11px;">(Ctrl + rueda para zoom)</span>
  </div>

  <div id="vp_{uid}" style="height:calc(100% - {toolbar_h}px); overflow:auto; background:#fff;">
    <div id="content_{uid}" data-basew="{base_w}" data-baseh="{base_h}"
         style="width:{base_w}px; height:{base_h}px;">  
    <svg id="svg_{uid}" xmlns="http://www.w3.org/2000/svg"
     width="{base_w}" height="{base_h}"
     viewBox="0 0 {base_w} {base_h}"
     preserveAspectRatio="xMinYMin meet"
     style="display:block; background:#ffffff;">
""".strip())

    parts.append("""
<defs>
  <marker id="arrow" viewBox="0 0 10 10" refX="10" refY="5" markerWidth="7" markerHeight="7" orient="auto-start-reverse">
    <path d="M 0 0 L 10 5 L 0 10 z" fill="#555"></path>
  </marker>
</defs>
""".strip())

    # edges
    for s, d, _ in edges:
        if s not in x_level or d not in x_level:
            continue
        x1 = x_level[s] + node_w
        y1 = y_pos[s] + node_h / 2
        x2 = x_level[d]
        y2 = y_pos[d] + node_h / 2
        mx = (x1 + x2) / 2
        parts.append(
            f'<path d="M {x1} {y1} C {mx} {y1}, {mx} {y2}, {x2} {y2}" '
            f'stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrow)"/>'
        )

    # nodes
    for n in sorted(nodes, key=lambda n: (x_level.get(n, 0), y_pos.get(n, 0))):
        x, y = x_level[n], y_pos[n]
        parts.append(
            f'<rect x="{x}" y="{y}" rx="10" ry="10" width="{node_w}" height="{node_h}" '
            f'fill="{zone_color(n)}" stroke="#333" stroke-width="1.2"/>'
        )

    # labels
    for n in sorted(nodes, key=lambda n: (x_level.get(n, 0), y_pos.get(n, 0))):
        x, y = x_level[n], y_pos[n]
        nid = node_ids.get(n, "")
        parts.append(f'<text x="{x+12}" y="{y+28}" font-size="14" fill="#111" font-weight="700">{svg_escape(nid)}</text>')
        lines = textwrap.wrap(svg_escape(n), width=34)
        for j, line in enumerate(lines[:2]):
            parts.append(f'<text x="{x+12}" y="{y+50 + j*16}" font-size="12" fill="#111">{line}</text>')

    parts.append(f"""
      </svg>
    </div>
  </div>
</div>

<script>
(function() {{
  var scale = 1.0;
  var minS = 0.3, maxS = 3.0, step = 0.1;

  var vp = document.getElementById("vp_{uid}");
  var content = document.getElementById("content_{uid}");
  var svg = document.getElementById("svg_{uid}");
  var lbl = document.getElementById("zlbl_{uid}");

  var baseW = parseFloat(content.getAttribute("data-basew")) || 1000;
  var baseH = parseFloat(content.getAttribute("data-baseh")) || 600;

  function clamp(v) {{ return Math.max(minS, Math.min(maxS, v)); }}

  function apply() {{
    var w = Math.round(baseW * scale);
    var h = Math.round(baseH * scale);

    content.style.width = w + "px";
    content.style.height = h + "px";
    svg.setAttribute("width", w);
    svg.setAttribute("height", h);

    if(lbl) lbl.textContent = Math.round(scale * 100) + "%";
  }}

  function zoomAt(deltaScale, clientX, clientY) {{
    if(!vp) return;

    var rect = vp.getBoundingClientRect();
    var x = clientX - rect.left;
    var y = clientY - rect.top;

    // punto actual en coordenadas de scroll (antes del zoom)
    var prev = scale;
    var scrollX = vp.scrollLeft + x;
    var scrollY = vp.scrollTop  + y;

    scale = clamp(scale * deltaScale);
    apply();

    // mantener el punto bajo el mouse “fijo”
    var ratio = scale / prev;
    vp.scrollLeft = Math.round(scrollX * ratio - x);
    vp.scrollTop  = Math.round(scrollY * ratio - y);
  }}

  // Botones
  var zin = document.getElementById("zin_{uid}");
  var zout = document.getElementById("zout_{uid}");
  var zreset = document.getElementById("zreset_{uid}");

  if(zin) zin.addEventListener("click", function() {{
    zoomAt(1.1, vp.getBoundingClientRect().left + 10, vp.getBoundingClientRect().top + 10);
  }});
  if(zout) zout.addEventListener("click", function() {{
    zoomAt(0.9, vp.getBoundingClientRect().left + 10, vp.getBoundingClientRect().top + 10);
  }});
  if(zreset) zreset.addEventListener("click", function() {{
    scale = 1.0; apply();
  }});

  // Ctrl + rueda (recomendado para no romper el scroll normal)
  if(vp) vp.addEventListener("wheel", function(e) {{
    if(!(e.ctrlKey || e.metaKey)) return;   // sin Ctrl = scroll normal
    e.preventDefault();
    var ds = (e.deltaY < 0) ? 1.1 : 0.9;
    zoomAt(ds, e.clientX, e.clientY);
  }}, {{ passive:false }});

  apply();
}})();
</script>
""".strip())

    return "\n".join(parts)


def build_canvas_html(nodes, edges, level_map, kind, node_ids, uid="g1", viewport_h=320):
    dx = 350
    dy = 140
    node_w, node_h = 270, 74

    max_level = max(level_map.values()) if level_map else 0

    if kind == "upstream":
        x_level = {n: (max_level - level_map.get(n, max_level)) * dx for n in nodes}
    else:
        x_level = {n: level_map.get(n, 0) * dx for n in nodes}

    buckets = defaultdict(list)
    for n in nodes:
        buckets[level_map.get(n, 0)].append(n)
    for k in buckets:
        buckets[k].sort(key=lambda n: n.lower())

    y_pos = {}
    for k, ns in buckets.items():
        for i, n in enumerate(ns):
            y_pos[n] = 60 + i * dy

    base_w = (max_level + 1) * dx + 520
    base_h = (max(y_pos.values()) if y_pos else 0) + 260

    toolbar_h = 38

    # Preparamos datos para JS
    # nodes_js: [{id, name, x, y, w, h, color}]
    nodes_list = []
    for n in nodes:
        nodes_list.append({
            "name": n,
            "id": node_ids.get(n, ""),
            "x": int(x_level[n]),
            "y": int(y_pos[n]),
            "w": int(node_w),
            "h": int(node_h),
            "color": zone_color(n)
        })

    edges_list = []
    for s, d, _ in edges:
        if s not in x_level or d not in x_level:
            continue
        edges_list.append({
            "sx": int(x_level[s] + node_w),
            "sy": int(y_pos[s] + node_h/2),
            "tx": int(x_level[d]),
            "ty": int(y_pos[d] + node_h/2)
        })

    import json
    nodes_json = json.dumps(nodes_list)
    edges_json = json.dumps(edges_list)

    html = f"""
<div style="width:100%; height:{viewport_h}px; overflow:hidden; background:#ffffff; font-family:Segoe UI, Arial;">
  <div style="display:flex; gap:8px; align-items:center; padding:6px 8px; height:{toolbar_h}px; box-sizing:border-box; border-bottom:1px solid #ddd;">
    <button id="zin_{uid}" style="padding:4px 10px;">zoom in</button>
    <button id="zout_{uid}" style="padding:4px 10px;">zoom out</button>
    <button id="zreset_{uid}" style="padding:4px 10px;">Reset</button>
    <span id="zlbl_{uid}" style="margin-left:8px; color:#444; font-size:12px;">100%</span>
    <span style="margin-left:10px; color:#777; font-size:11px;">(Ctrl + rueda para zoom)</span>
  </div>

  <div id="vp_{uid}" style="height:calc(100% - {toolbar_h}px); overflow:auto; background:#fff;">
    <!-- Este DIV es el "mundo" scrolleable: su tamaño cambia con el zoom -->
    <div id="world_{uid}" data-basew="{base_w}" data-baseh="{base_h}"
         style="width:{base_w}px; height:{base_h}px; position:relative;">
      <canvas id="cv_{uid}" width="{base_w}" height="{base_h}"
              style="display:block; background:#ffffff;"></canvas>
    </div>
  </div>
</div>

<script>
(function() {{
  var nodes = {nodes_json};
  var edges = {edges_json};

  var scale = 1.0;
  var minS = 0.3, maxS = 3.0;

  var vp = document.getElementById("vp_{uid}");
  var world = document.getElementById("world_{uid}");
  var canvas = document.getElementById("cv_{uid}");
  var ctx = canvas.getContext("2d");
  var lbl = document.getElementById("zlbl_{uid}");

  var baseW = parseFloat(world.getAttribute("data-basew")) || 1000;
  var baseH = parseFloat(world.getAttribute("data-baseh")) || 600;

  function clamp(v) {{ return Math.max(minS, Math.min(maxS, v)); }}

  function setWorldSize() {{
    var w = Math.round(baseW * scale);
    var h = Math.round(baseH * scale);
    world.style.width = w + "px";
    world.style.height = h + "px";
    canvas.width = w;
    canvas.height = h;
    if(lbl) lbl.textContent = Math.round(scale * 100) + "%";
  }}

  function drawRoundedRect(x,y,w,h,r) {{
    ctx.beginPath();
    ctx.moveTo(x+r, y);
    ctx.lineTo(x+w-r, y);
    ctx.quadraticCurveTo(x+w, y, x+w, y+r);
    ctx.lineTo(x+w, y+h-r);
    ctx.quadraticCurveTo(x+w, y+h, x+w-r, y+h);
    ctx.lineTo(x+r, y+h);
    ctx.quadraticCurveTo(x, y+h, x, y+h-r);
    ctx.lineTo(x, y+r);
    ctx.quadraticCurveTo(x, y, x+r, y);
    ctx.closePath();
  }}

  function drawArrow(x1,y1,x2,y2) {{
    // Curva bezier similar al SVG
    var mx = (x1 + x2) / 2;

    ctx.strokeStyle = "#555";
    ctx.lineWidth = 2;
    ctx.beginPath();
    ctx.moveTo(x1, y1);
    ctx.bezierCurveTo(mx, y1, mx, y2, x2, y2);
    ctx.stroke();

    // Flecha (triángulo) al final
    var angle = Math.atan2(y2 - y1, x2 - x1);
    var headLen = 10;
    var a1 = angle + Math.PI/7;
    var a2 = angle - Math.PI/7;

    ctx.fillStyle = "#555";
    ctx.beginPath();
    ctx.moveTo(x2, y2);
    ctx.lineTo(x2 - headLen*Math.cos(a1), y2 - headLen*Math.sin(a1));
    ctx.lineTo(x2 - headLen*Math.cos(a2), y2 - headLen*Math.sin(a2));
    ctx.closePath();
    ctx.fill();
  }}

  function wrapText(text, maxChars) {{
    // simple wrap por caracteres, similar al python
    var out = [];
    var s = text || "";
    while(s.length > maxChars) {{
      out.push(s.slice(0, maxChars));
      s = s.slice(maxChars);
    }}
    if(s.length) out.push(s);
    return out;
  }}

  function render() {{
    ctx.clearRect(0,0,canvas.width,canvas.height);

    // Edges primero
    for(var i=0;i<edges.length;i++) {{
      var e = edges[i];
      drawArrow(e.sx*scale, e.sy*scale, e.tx*scale, e.ty*scale);
    }}

    // Nodes
    for(var j=0;j<nodes.length;j++) {{
      var n = nodes[j];
      var x = n.x*scale, y = n.y*scale, w = n.w*scale, h = n.h*scale;

      // rect
      ctx.fillStyle = n.color;
      ctx.strokeStyle = "#333";
      ctx.lineWidth = 1.2;
      drawRoundedRect(x,y,w,h,10*scale);
      ctx.fill();
      ctx.stroke();

      // texts
      ctx.fillStyle = "#111";
      ctx.font = (14*scale).toFixed(0) + "px Segoe UI, Arial";
      ctx.fillText(n.id, x + 12*scale, y + 28*scale);

      ctx.font = (12*scale).toFixed(0) + "px Segoe UI, Arial";
      var lines = wrapText(n.name, 34);
      for(var k=0;k<Math.min(2, lines.length);k++) {{
        ctx.fillText(lines[k], x + 12*scale, y + (50 + k*16)*scale);
      }}
    }}
  }}

  function apply() {{
    setWorldSize();
    render();
  }}

  function zoomAt(mult, clientX, clientY) {{
    if(!vp) return;
    var rect = vp.getBoundingClientRect();
    var x = clientX - rect.left;
    var y = clientY - rect.top;

    var prev = scale;
    var scrollX = vp.scrollLeft + x;
    var scrollY = vp.scrollTop  + y;

    scale = clamp(scale * mult);
    apply();

    var ratio = scale / prev;
    vp.scrollLeft = Math.round(scrollX * ratio - x);
    vp.scrollTop  = Math.round(scrollY * ratio - y);
  }}

  // Botones
  var zin = document.getElementById("zin_{uid}");
  var zout = document.getElementById("zout_{uid}");
  var zreset = document.getElementById("zreset_{uid}");

  if(zin) zin.addEventListener("click", function() {{
    var r = vp.getBoundingClientRect();
    zoomAt(1.1, r.left + 30, r.top + 30);
  }});
  if(zout) zout.addEventListener("click", function() {{
    var r = vp.getBoundingClientRect();
    zoomAt(0.9, r.left + 30, r.top + 30);
  }});
  if(zreset) zreset.addEventListener("click", function() {{
    scale = 1.0; apply();
  }});

  // Ctrl + rueda
  if(vp) vp.addEventListener("wheel", function(e) {{
    if(!(e.ctrlKey || e.metaKey)) return;
    e.preventDefault();
    var ds = (e.deltaY < 0) ? 1.1 : 0.9;
    zoomAt(ds, e.clientX, e.clientY);
  }}, {{ passive:false }});

  apply();
}})();
</script>
""".strip()

    return html




""" Versioon sin limite en zoom """

def build_canvas_html(nodes, edges, level_map, kind, node_ids, uid="g1", viewport_h=320, objetivo_fqn=""):
    dx = 350
    dy = 140
    node_w, node_h = 270, 74

    max_level = max(level_map.values()) if level_map else 0

    if kind == "upstream":
        x_level = {n: (max_level - level_map.get(n, max_level)) * dx for n in nodes}
    else:
        x_level = {n: level_map.get(n, 0) * dx for n in nodes}

    buckets = defaultdict(list)
    for n in nodes:
        buckets[level_map.get(n, 0)].append(n)
    for k in buckets:
        buckets[k].sort(key=lambda n: n.lower())

    y_pos = {}
    for k, ns in buckets.items():
        for i, n in enumerate(ns):
            y_pos[n] = 60 + i * dy

    base_w = (max_level + 1) * dx + 520
    base_h = (max(y_pos.values()) if y_pos else 0) + 260

    toolbar_h = 42

    # Nodos para JS
    nodes_list = []
    for n in nodes:
        nodes_list.append({
            "name": n,
            "id": node_ids.get(n, ""),
            "x": float(x_level[n]),
            "y": float(y_pos[n]),
            "w": float(node_w),
            "h": float(node_h),
            "color": zone_color(n)
        })

    # Edges para JS (con src/dst para poder resaltar)
    edges_list = []
    for s, d, _ in edges:
        if s not in x_level or d not in x_level:
            continue
        edges_list.append({
            "src": s,
            "dst": d,
            "sx": float(x_level[s] + node_w),
            "sy": float(y_pos[s] + node_h/2),
            "tx": float(x_level[d]),
            "ty": float(y_pos[d] + node_h/2)
        })

    nodes_json = json.dumps(nodes_list)
    edges_json = json.dumps(edges_list)
    objetivo_json = json.dumps(objetivo_fqn)

    html = f"""
<div id="wrap_{uid}" style="position:relative; width:100%; height:{viewport_h}px; overflow:hidden; background:#ffffff; font-family:Segoe UI, Arial;">
  <div style="display:flex; gap:8px; align-items:center; padding:6px 8px; height:{toolbar_h}px; box-sizing:border-box; border-bottom:1px solid #ddd;">
    <button id="zin_{uid}" style="padding:4px 10px;">Acercar (+)</button>
    <button id="zout_{uid}" style="padding:4px 10px;">Alejar (-)</button>
    <button id="zreset_{uid}" style="padding:4px 10px;">Restablecer</button>
    <span id="zlbl_{uid}" style="margin-left:8px; color:#444; font-size:12px;">0%</span>
    <span style="margin-left:10px; color:#777; font-size:11px;">Escala relativa (base = 0%) | Ctrl + rueda para zoom</span>
  </div>

  <div id="vp_{uid}" style="height:calc(100% - {toolbar_h}px); overflow:auto; background:#fff;">
    <div id="world_{uid}" data-basew="{base_w}" data-baseh="{base_h}"
         style="width:{base_w}px; height:{base_h}px; position:relative;">
      <canvas id="cv_{uid}" width="{base_w}" height="{base_h}"
              style="display:block; background:#ffffff;"></canvas>
    </div>
  </div>

  <!-- POPUP -->
  <div id="info_{uid}"
       style="position:absolute; top:{toolbar_h + 8}px; right:10px;
              width:360px; max-height:60%;
              overflow:auto;
              background:#fff; border:1px solid #bbb; border-radius:8px;
              box-shadow:0 6px 20px rgba(0,0,0,.12);
              padding:10px 12px;
              display:none;
              z-index:9999;">
    <div style="font-weight:700; margin-bottom:6px;">Detalle del nodo</div>
    <div style="font-size:12px; color:#555; margin-bottom:8px;">
      <div><b>Tabla objetivo:</b> <span id="obj_{uid}"></span></div>
      <div style="margin-top:4px;"><b>Nodo seleccionado:</b> <span id="sel_{uid}"></span></div>
    </div>
    <div style="font-weight:700; margin:8px 0 6px 0;">Tablas origen directas</div>
    <ul id="orig_{uid}" style="margin:0; padding-left:18px; font-size:12px;"></ul>
    <div style="margin-top:8px; font-size:11px; color:#777;">
      * Los enlaces origen → nodo se resaltan en <span style="color:#c0392b;font-weight:700;">rojo</span>.
    </div>
  </div>
</div>

<script>
(function() {{
  var nodes = {nodes_json};
  var edges = {edges_json};
  var objetivo = {objetivo_json};

  var scale = 1.0;
  var minS = 0.3, maxS = 3.0;

  var vp = document.getElementById("vp_{uid}");
  var world = document.getElementById("world_{uid}");
  var canvas = document.getElementById("cv_{uid}");
  var ctx = canvas.getContext("2d");
  var lbl = document.getElementById("zlbl_{uid}");

  var info = document.getElementById("info_{uid}");
  var objSpan = document.getElementById("obj_{uid}");
  var selSpan = document.getElementById("sel_{uid}");
  var origUl = document.getElementById("orig_{uid}");

  // Estado de selección
  var selectedName = null;
  var selectedOrigins = new Set();

  var baseW = parseFloat(world.getAttribute("data-basew")) || 1000;
  var baseH = parseFloat(world.getAttribute("data-baseh")) || 600;

  function clamp(v) {{ return Math.max(minS, Math.min(maxS, v)); }}

  function updateScaleLabel() {{
    if(!lbl) return;
    // Escala relativa: 1.0 = 0%
    var pct = Math.round((scale - 1) * 100);
    lbl.textContent = (pct > 0 ? "+" : "") + pct + "%";
  }}

  function setWorldSize() {{
    var w = Math.round(baseW * scale);
    var h = Math.round(baseH * scale);

    world.style.width = w + "px";
    world.style.height = h + "px";

    // Canvas render real al tamaño escalado
    canvas.width = w;
    canvas.height = h;

    updateScaleLabel();
  }}

  function drawRoundedRect(x,y,w,h,r) {{
    ctx.beginPath();
    ctx.moveTo(x+r, y);
    ctx.lineTo(x+w-r, y);
    ctx.quadraticCurveTo(x+w, y, x+w, y+r);
    ctx.lineTo(x+w, y+h-r);
    ctx.quadraticCurveTo(x+w, y+h, x+w-r, y+h);
    ctx.lineTo(x+r, y+h);
    ctx.quadraticCurveTo(x, y+h, x, y+h-r);
    ctx.lineTo(x, y+r);
    ctx.quadraticCurveTo(x, y, x+r, y);
    ctx.closePath();
  }}

  function drawArrow(x1,y1,x2,y2, color, width) {{
    var mx = (x1 + x2) / 2;

    ctx.strokeStyle = color;
    ctx.lineWidth = width;
    ctx.beginPath();
    ctx.moveTo(x1, y1);
    ctx.bezierCurveTo(mx, y1, mx, y2, x2, y2);
    ctx.stroke();

    // Flecha (triángulo)
    var angle = Math.atan2(y2 - y1, x2 - x1);
    var headLen = Math.max(8, 10 * (width/2));
    var a1 = angle + Math.PI/7;
    var a2 = angle - Math.PI/7;

    ctx.fillStyle = color;
    ctx.beginPath();
    ctx.moveTo(x2, y2);
    ctx.lineTo(x2 - headLen*Math.cos(a1), y2 - headLen*Math.sin(a1));
    ctx.lineTo(x2 - headLen*Math.cos(a2), y2 - headLen*Math.sin(a2));
    ctx.closePath();
    ctx.fill();
  }}

  function wrapTextSimple(text, maxChars) {{
    var out = [];
    var s = (text || "");

    while (s.length > maxChars) {{
      // intentar cortar por separadores para que se vea mejor
      var cut = -1;
      var sub = s.slice(0, maxChars + 1);
      var seps = [".", "_", "-", " "];
      for (var i = 0; i < seps.length; i++) {{
        var p = sub.lastIndexOf(seps[i]);
        if (p > 8) cut = Math.max(cut, p + 1);
      }}
      if (cut === -1) cut = maxChars;

      out.push(s.slice(0, cut));
      s = s.slice(cut);
    }}
    if (s.length) out.push(s);

    return out;
  }}

  function render() {{
    ctx.clearRect(0, 0, canvas.width, canvas.height);

    // Edges primero
    for (var i = 0; i < edges.length; i++) {{
      var e = edges[i];
      var isRed = (selectedName && e.dst === selectedName && selectedOrigins.has(e.src));
      var color = isRed ? "#c0392b" : "#555";
      var w = isRed ? 3 : 2;
      drawArrow(e.sx*scale, e.sy*scale, e.tx*scale, e.ty*scale, color, w);
    }}

    // Nodes
    for (var j = 0; j < nodes.length; j++) {{
      var n = nodes[j];
      var x = n.x*scale, y = n.y*scale, w = n.w*scale, h = n.h*scale;

      ctx.fillStyle = n.color;
      ctx.strokeStyle = "#333";
      ctx.lineWidth = 1.2;
      drawRoundedRect(x, y, w, h, 10*scale);
      ctx.fill();
      ctx.stroke();

      // texto id
      ctx.fillStyle = "#111";
      ctx.font = "700 " + Math.max(10, Math.round(14*scale)) + "px Segoe UI, Arial";
      ctx.fillText(n.id, x + 12*scale, y + 28*scale);

      // nombre tabla (máximo 2 líneas)
      ctx.font = Math.max(9, Math.round(12*scale)) + "px Segoe UI, Arial";
      var lines = wrapTextSimple(n.name, 34);
      for (var k = 0; k < Math.min(2, lines.length); k++) {{
        ctx.fillText(lines[k], x + 12*scale, y + (50 + k*16)*scale);
      }}
    }}
  }}

  function apply() {{
    setWorldSize();
    render();
  }}

  function zoomAt(mult, clientX, clientY) {{
    if (!vp) return;

    var rect = vp.getBoundingClientRect();
    var x = clientX - rect.left;
    var y = clientY - rect.top;

    // posición del puntero en coordenadas del "mundo" antes del zoom
    var prev = scale;
    var scrollX = vp.scrollLeft + x;
    var scrollY = vp.scrollTop + y;

    scale = clamp(scale * mult);
    apply();

    // mantener el punto bajo el mouse
    var ratio = scale / prev;
    vp.scrollLeft = Math.round(scrollX * ratio - x);
    vp.scrollTop  = Math.round(scrollY * ratio - y);
  }}

  function openPopup(nodeName) {{
    selectedName = nodeName;
    selectedOrigins = new Set();

    for (var i = 0; i < edges.length; i++) {{
      if (edges[i].dst === selectedName) selectedOrigins.add(edges[i].src);
    }}

    if (objSpan) objSpan.textContent = objetivo || "";
    if (selSpan) selSpan.textContent = selectedName || "";

    if (origUl) {{
      origUl.innerHTML = "";
      var arr = Array.from(selectedOrigins).sort();

      if (arr.length === 0) {{
        var li0 = document.createElement("li");
        li0.textContent = "(sin orígenes directos en este grafo)";
        origUl.appendChild(li0);
      }} else {{
        for (var k = 0; k < arr.length; k++) {{
          var li = document.createElement("li");
          li.textContent = arr[k];
          origUl.appendChild(li);
        }}
      }}
    }}

    if (info) info.style.display = "block";
    render();
  }}

  function closePopup() {{
    selectedName = null;
    selectedOrigins = new Set();
    if (info) info.style.display = "none";
    render();
  }}

  function findNodeAt(px, py) {{
    // px,py están en canvas escalado; convertir a coordenadas de mundo
    var mx = px / scale;
    var my = py / scale;

    for (var i = nodes.length - 1; i >= 0; i--) {{
      var n = nodes[i];
      if (mx >= n.x && mx <= (n.x + n.w) && my >= n.y && my <= (n.y + n.h)) {{
        return n.name;
      }}
    }}
    return null;
  }}

  // Click sobre canvas: abre/cambia popup o cierra si clic fuera de nodo
  canvas.addEventListener("click", function(e) {{
    var name = findNodeAt(e.offsetX, e.offsetY);
    if (name) {{
      openPopup(name);
    }} else {{
      closePopup();
    }}
  }});

  // Evitar que click dentro del popup cierre
  if (info) {{
    info.addEventListener("click", function(e) {{
      e.stopPropagation();
    }});
  }}

  // Click fuera del popup (pero dentro del wrap) => cerrar
  var wrap = document.getElementById("wrap_{uid}");
  if (wrap) {{
    wrap.addEventListener("click", function(e) {{
      if (info && info.style.display === "block") {{
        if (!info.contains(e.target) && e.target !== canvas) {{
          closePopup();
        }}
      }}
    }});
  }}

  // Botones
  var zin = document.getElementById("zin_{uid}");
  var zout = document.getElementById("zout_{uid}");
  var zreset = document.getElementById("zreset_{uid}");

  if (zin) zin.addEventListener("click", function() {{
    var r = vp.getBoundingClientRect();
    zoomAt(1.1, r.left + 30, r.top + 30);
  }});

  if (zout) zout.addEventListener("click", function() {{
    var r = vp.getBoundingClientRect();
    zoomAt(0.9, r.left + 30, r.top + 30);
  }});

  if (zreset) zreset.addEventListener("click", function() {{
    scale = 1.0;
    apply();
  }});

  // Ctrl + rueda para zoom (scroll normal sin Ctrl)
  if (vp) {{
    vp.addEventListener("wheel", function(e) {{
      if (!(e.ctrlKey || e.metaKey)) return;
      e.preventDefault();
      var ds = (e.deltaY < 0) ? 1.1 : 0.9;
      zoomAt(ds, e.clientX, e.clientY);
    }}, {{ passive:false }});
  }}

  apply();
}})();
</script>
""".strip()

    return html

def build_canvas_html(nodes, edges, level_map, kind, node_ids, uid="g1", viewport_h=320, objetivo_fqn=""):
    dx = 350
    dy = 140
    node_w, node_h = 270, 74

    max_level = max(level_map.values()) if level_map else 0

    if kind == "upstream":
        x_level = {n: (max_level - level_map.get(n, max_level)) * dx for n in nodes}
    else:
        x_level = {n: level_map.get(n, 0) * dx for n in nodes}

    buckets = defaultdict(list)
    for n in nodes:
        buckets[level_map.get(n, 0)].append(n)
    for k in buckets:
        buckets[k].sort(key=lambda n: n.lower())

    y_pos = {}
    for k, ns in buckets.items():
        for i, n in enumerate(ns):
            y_pos[n] = 60 + i * dy

    base_w = (max_level + 1) * dx + 520
    base_h = (max(y_pos.values()) if y_pos else 0) + 260

    toolbar_h = 38

    # Nodos para JS (incluye name/id/rect/color)
    nodes_list = []
    for n in nodes:
        nodes_list.append({
            "name": n,
            "id": node_ids.get(n, ""),
            "x": float(x_level[n]),
            "y": float(y_pos[n]),
            "w": float(node_w),
            "h": float(node_h),
            "color": zone_color(n)
        })

    # Edges para JS (incluye src/dst por nombre y coords)
    edges_list = []
    for s, d, _ in edges:
        if s not in x_level or d not in x_level:
            continue
        edges_list.append({
            "src": s,
            "dst": d,
            "sx": float(x_level[s] + node_w),
            "sy": float(y_pos[s] + node_h/2),
            "tx": float(x_level[d]),
            "ty": float(y_pos[d] + node_h/2)
        })

    nodes_json = json.dumps(nodes_list)
    edges_json = json.dumps(edges_list)
    objetivo_json = json.dumps(objetivo_fqn)

    html = f"""
<div id="wrap_{uid}" style="position:relative; width:100%; height:{viewport_h}px; overflow:hidden; background:#ffffff; font-family:Segoe UI, Arial;">
  <div style="display:flex; gap:8px; align-items:center; padding:6px 8px; height:{toolbar_h}px; box-sizing:border-box; border-bottom:1px solid #ddd;">
    <button id="zin_{uid}" style="padding:4px 10px;">zoom in</button>
    <button id="zout_{uid}" style="padding:4px 10px;">zoom out</button>
    <button id="zreset_{uid}" style="padding:4px 10px;">Reset</button>
    <span id="zlbl_{uid}" style="margin-left:8px; color:#444; font-size:12px;">100%</span>
    <span style="margin-left:10px; color:#777; font-size:11px;">(Ctrl + rueda para zoom)</span>
  </div>

  <div id="vp_{uid}" style="height:calc(100% - {toolbar_h}px); overflow:auto; background:#fff;">
    <div id="world_{uid}" data-basew="{base_w}" data-baseh="{base_h}"
         style="width:{base_w}px; height:{base_h}px; position:relative;">
      <canvas id="cv_{uid}" width="{base_w}" height="{base_h}"
              style="display:block; background:#ffffff;"></canvas>
    </div>
  </div>

  <!-- POPUP -->
  <div id="info_{uid}"
       style="position:absolute; top:{toolbar_h + 8}px; right:10px;
              width:360px; max-height:60%;
              overflow:auto;
              background:#fff; border:1px solid #bbb; border-radius:8px;
              box-shadow:0 6px 20px rgba(0,0,0,.12);
              padding:10px 12px;
              display:none;
              z-index:9999;">
    <div style="font-weight:700; margin-bottom:6px;">Detalle del nodo</div>
    <div style="font-size:12px; color:#555; margin-bottom:8px;">
      <div><b>Tabla objetivo:</b> <span id="obj_{uid}"></span></div>
      <div style="margin-top:4px;"><b>Nodo seleccionado:</b> <span id="sel_{uid}"></span></div>
    </div>
    <div style="font-weight:700; margin:8px 0 6px 0;">Tablas origen directas</div>
    <ul id="orig_{uid}" style="margin:0; padding-left:18px; font-size:12px;"></ul>
    <div style="margin-top:8px; font-size:11px; color:#777;">
      * Los enlaces origen a nodo se resaltan en <span style="color:#c0392b;font-weight:700;">rojo</span>.
    </div>
  </div>
</div>

<script>
(function() {{
  var nodes = {nodes_json};
  var edges = {edges_json};
  var objetivo = {objetivo_json};

  var scale = 1.0;
  var minS = 0.05, maxS = 3.0;

  var vp = document.getElementById("vp_{uid}");
  var world = document.getElementById("world_{uid}");
  var canvas = document.getElementById("cv_{uid}");
  var ctx = canvas.getContext("2d");
  var lbl = document.getElementById("zlbl_{uid}");

  var info = document.getElementById("info_{uid}");
  var objSpan = document.getElementById("obj_{uid}");
  var selSpan = document.getElementById("sel_{uid}");
  var origUl = document.getElementById("orig_{uid}");

  // Estado de selección
  var selectedName = null;
  var selectedOrigins = new Set(); // src directas del seleccionado

  var baseW = parseFloat(world.getAttribute("data-basew")) || 1000;
  var baseH = parseFloat(world.getAttribute("data-baseh")) || 600;

  function clamp(v) {{ return Math.max(minS, Math.min(maxS, v)); }}

  function setWorldSize() {{
    var w = Math.round(baseW * scale);
    var h = Math.round(baseH * scale);
    world.style.width = w + "px";
    world.style.height = h + "px";
    canvas.width = w;
    canvas.height = h;
    if(lbl) lbl.textContent = Math.round(scale * 100) + "%";
  }}

  function drawRoundedRect(x,y,w,h,r) {{
    ctx.beginPath();
    ctx.moveTo(x+r, y);
    ctx.lineTo(x+w-r, y);
    ctx.quadraticCurveTo(x+w, y, x+w, y+r);
    ctx.lineTo(x+w, y+h-r);
    ctx.quadraticCurveTo(x+w, y+h, x+w-r, y+h);
    ctx.lineTo(x+r, y+h);
    ctx.quadraticCurveTo(x, y+h, x, y+h-r);
    ctx.lineTo(x, y+r);
    ctx.quadraticCurveTo(x, y, x+r, y);
    ctx.closePath();
  }}

  function drawArrow(x1,y1,x2,y2, color, width) {{
    var mx = (x1 + x2) / 2;
    ctx.strokeStyle = color;
    ctx.lineWidth = width;
    ctx.beginPath();
    ctx.moveTo(x1, y1);
    ctx.bezierCurveTo(mx, y1, mx, y2, x2, y2);
    ctx.stroke();

    // Flecha (triángulo)
    var angle = Math.atan2(y2 - y1, x2 - x1);
    var headLen = 10 * (width/2);
    var a1 = angle + Math.PI/7;
    var a2 = angle - Math.PI/7;

    ctx.fillStyle = color;
    ctx.beginPath();
    ctx.moveTo(x2, y2);
    ctx.lineTo(x2 - headLen*Math.cos(a1), y2 - headLen*Math.sin(a1));
    ctx.lineTo(x2 - headLen*Math.cos(a2), y2 - headLen*Math.sin(a2));
    ctx.closePath();
    ctx.fill();
  }}

  function wrapText(text, maxChars) {{
    var out = [];
    var s = (text || "");
    while(s.length > maxChars) {{
      out.push(s.slice(0, maxChars));
      s = s.slice(maxChars);
    }}
    if(s.length) out.push(s);
    return out;
  }}

  function render() {{
    ctx.clearRect(0,0,canvas.width,canvas.height);

    // Edges primero: si están seleccionadas, rojo
    for(var i=0;i<edges.length;i++) {{
      var e = edges[i];
      var isRed = (selectedName && e.dst === selectedName && selectedOrigins.has(e.src));
      var color = isRed ? "#c0392b" : "#555";
      var w = isRed ? 3 : 2;
      drawArrow(e.sx*scale, e.sy*scale, e.tx*scale, e.ty*scale, color, w);
    }}

    // Nodes
    for(var j=0;j<nodes.length;j++) {{
      var n = nodes[j];
      var x = n.x*scale, y = n.y*scale, w = n.w*scale, h = n.h*scale;

      ctx.fillStyle = n.color;
      ctx.strokeStyle = "#333";
      ctx.lineWidth = 1.2;
      drawRoundedRect(x,y,w,h,10*scale);
      ctx.fill();
      ctx.stroke();

      // Texto
      ctx.fillStyle = "#111";
      ctx.font = (14*scale).toFixed(0) + "px Segoe UI, Arial";
      ctx.fillText(n.id, x + 12*scale, y + 28*scale);

      ctx.font = (12*scale).toFixed(0) + "px Segoe UI, Arial";
      var lines = wrapText(n.name, 34);
      for(var k=0;k<Math.min(2, lines.length);k++) {{
        ctx.fillText(lines[k], x + 12*scale, y + (50 + k*16)*scale);
      }}
    }}
  }}

  function apply() {{
    setWorldSize();
    render();
  }}

  function zoomAt(mult, clientX, clientY) {{
    if(!vp) return;
    var rect = vp.getBoundingClientRect();
    var x = clientX - rect.left;
    var y = clientY - rect.top;

    var prev = scale;
    var scrollX = vp.scrollLeft + x;
    var scrollY = vp.scrollTop  + y;

    scale = clamp(scale * mult);
    apply();

    var ratio = scale / prev;
    vp.scrollLeft = Math.round(scrollX * ratio - x);
    vp.scrollTop  = Math.round(scrollY * ratio - y);
  }}

  function openPopup(nodeName) {{
    selectedName = nodeName;
    selectedOrigins = new Set();

    // Orígenes directos: edges src -> selectedName
    for(var i=0;i<edges.length;i++) {{
      if(edges[i].dst === selectedName) selectedOrigins.add(edges[i].src);
    }}

    // Pintar popup
    if(objSpan) objSpan.textContent = objetivo || "";
    if(selSpan) selSpan.textContent = selectedName || "";

    if(origUl) {{
      origUl.innerHTML = "";
      var arr = Array.from(selectedOrigins).sort();
      if(arr.length === 0) {{
        var li = document.createElement("li");
        li.textContent = "(sin origenes directos en este grafo)";
        origUl.appendChild(li);
      }} else {{
        for(var k=0;k<arr.length;k++) {{
          var li = document.createElement("li");
          li.textContent = arr[k];
          origUl.appendChild(li);
        }}
      }}
    }}

    if(info) info.style.display = "block";
    render(); // para resaltar enlaces
  }}

  function closePopup() {{
    selectedName = null;
    selectedOrigins = new Set();
    if(info) info.style.display = "none";
    render(); // quitar resaltado
  }}

  // Hit-test: click sobre nodo
  function findNodeAt(px, py) {{
    // px, py vienen en coordenadas CANVAS (ya escaladas), pasamos a mundo
    var mx = px / scale;
    var my = py / scale;

    // iterar al revés para priorizar el “último dibujado”
    for(var i=nodes.length-1; i>=0; i--) {{
      var n = nodes[i];
      if(mx >= n.x && mx <= n.x + n.w && my >= n.y && my <= n.y + n.h) {{
        return n.name;
      }}
    }}
    return null;
  }}

  // Clicks
  canvas.addEventListener("click", function(e) {{
    var name = findNodeAt(e.offsetX, e.offsetY);
    if(name) {{
      // Selecciona nodo (cierra anterior y abre el nuevo automáticamente)
      openPopup(name);
    }} else {{
      // Click fuera de nodos: cerrar
      closePopup();
    }}
  }});

  // Click dentro del popup NO lo cierra
  info.addEventListener("click", function(e) {{
    e.stopPropagation();
  }});

  // Click fuera del popup (pero dentro del contenedor) lo cierra
  document.getElementById("wrap_{uid}").addEventListener("click", function(e) {{
    // si el click fue en el canvas, ya se manejó arriba
    // si fue en otra zona y el popup está abierto, cierra
    if(info && info.style.display === "block") {{
      // si el click no fue dentro del popup
      if(!info.contains(e.target) && e.target !== canvas) {{
        closePopup();
      }}
    }}
  }});

  // Botones
  var zin = document.getElementById("zin_{uid}");
  var zout = document.getElementById("zout_{uid}");
  var zreset = document.getElementById("zreset_{uid}");

  if(zin) zin.addEventListener("click", function() {{
    var r = vp.getBoundingClientRect();
    zoomAt(1.1, r.left + 30, r.top + 30);
  }});
  if(zout) zout.addEventListener("click", function() {{
    var r = vp.getBoundingClientRect();
    zoomAt(0.9, r.left + 30, r.top + 30);
  }});
  if(zreset) zreset.addEventListener("click", function() {{
    scale = 1.0; apply();
  }});

  // Ctrl + rueda
  vp.addEventListener("wheel", function(e) {{
    if(!(e.ctrlKey || e.metaKey)) return;
    e.preventDefault();
    var ds = (e.deltaY < 0) ? 1.1 : 0.9;
    zoomAt(ds, e.clientX, e.clientY);
  }}, {{ passive:false }});

  apply();
}})();
</script>
""".strip()

    return html


# -----------------------------
# Objetivos:
#  A) todos los tabla_destino (upstream)
#  B) todas las tablas crudas s_* que aparezcan como tabla_origen (downstream)
# -----------------------------
dest_targets = sorted(insumo_base["tabla_destino"].unique(), key=lambda s: s.lower())
raw_sources = sorted({t for t in insumo_base["tabla_origen"].unique() if parse_schema(t).startswith("s_")}, key=lambda s: s.lower())

objectives = [("upstream", t) for t in dest_targets] + [("downstream", s) for s in raw_sources]

# -----------------------------
# Generar tb_linaje y tb_html
# -----------------------------
tb_linaje_rows = []
tb_html_rows = []

for kind, obj in objectives:
    if kind == "upstream":
        nodes, edges = upstream_closure(obj)
        level_map = levels_upstream(edges, obj)
    else:
        nodes, edges = downstream_closure(obj)
        level_map = levels_downstream(edges, obj)

    node_ids = assign_ids_per_objective(nodes, level_map, kind)

    uid = str(abs(hash(obj)) % 10_000_000)
    #html = build_svg_html(nodes, edges, level_map, kind, node_ids, uid=uid)
    #html = build_canvas_html(nodes, edges, level_map, kind, node_ids, uid=uid, viewport_h=320)
    html = build_canvas_html(nodes, edges, level_map, kind, node_ids, uid=uid, viewport_h=320, objetivo_fqn=obj)



    #html = build_svg_html(nodes, edges, level_map, kind, node_ids)

    # tb_linaje
    for s, d, qry in edges:
        tb_linaje_rows.append({
            "tabla_linaje_objetivo": obj,
            "id": node_ids.get(d, ""),
            "tabla_origen": s,
            "tabla_destino": d,
            "query": qry
        })

    tb_html_rows.append({"tabla_destino": obj, "html": html})

tb_linaje_v2 = pd.DataFrame(tb_linaje_rows)
tb_html_v2 = pd.DataFrame(tb_html_rows)

# -----------------------------
# Guardar CSVs
# -----------------------------
out_dir = "./resultados_linaje"
linaje_path = out_dir + "/tb_linaje_generado.csv"
html_path = out_dir +"/tb_html_generado.csv"

tb_linaje_v2.to_csv(linaje_path, index=False, quoting=csv.QUOTE_ALL)
tb_html_v2.to_csv(html_path, index=False, quoting=csv.QUOTE_ALL)

summary = pd.DataFrame({
    "objetivos_destino_upstream": [len(dest_targets)],
    "objetivos_crudos_downstream": [len(raw_sources)],
    "total_objetivos": [len(objectives)],
    "filas_tb_linaje": [len(tb_linaje_v2)],
    "filas_tb_html": [len(tb_html_v2)]
})

(str(linaje_path), str(html_path), summary)


('./resultados_linaje/tb_linaje_generado.csv',
 './resultados_linaje/tb_html_generado.csv',
    objetivos_destino_upstream  objetivos_crudos_downstream  total_objetivos  \
 0                          46                           20               66   
 
    filas_tb_linaje  filas_tb_html  
 0              818             66  )

In [10]:
import pandas as pd, csv, textwrap
from pathlib import Path
import json
from collections import defaultdict, deque

# -----------------------------
# Ejemplo de insumo_base (igual que antes)
# -----------------------------
insumo_rows = [
    # =========================
    # Fuentes crudas -> staging (20)
    # =========================
    ("s_core.clientes", "proceso_bipa_vpr.stg_clientes",
     "create table proceso_bipa_vpr.stg_clientes as select * from s_core.clientes;"),
    ("s_core.prestamos", "proceso_bipa_vpr.stg_prestamos",
     "create table proceso_bipa_vpr.stg_prestamos as select * from s_core.prestamos;"),
    ("s_core.pagos", "proceso_bipa_vpr.stg_pagos",
     "create table proceso_bipa_vpr.stg_pagos as select * from s_core.pagos;"),
    ("s_core.cartera", "proceso_riesgo.stg_cartera",
     "create table proceso_riesgo.stg_cartera as select * from s_core.cartera;"),
    ("s_core.sucursales", "proceso_bipa_vpr.stg_sucursales",
     "create table proceso_bipa_vpr.stg_sucursales as select * from s_core.sucursales;"),
    ("s_core.productos", "proceso_bipa_vpr.stg_productos",
     "create table proceso_bipa_vpr.stg_productos as select * from s_core.productos;"),
    ("s_core.tasas", "proceso_bipa_vpr.stg_tasas",
     "create table proceso_bipa_vpr.stg_tasas as select * from s_core.tasas;"),
    ("s_core.garantias", "proceso_bipa_vpr.stg_garantias",
     "create table proceso_bipa_vpr.stg_garantias as select * from s_core.garantias;"),
    ("s_core.transacciones", "proceso_bipa_vpr.stg_transacciones",
     "create table proceso_bipa_vpr.stg_transacciones as select * from s_core.transacciones;"),
    ("s_core.calendario", "proceso_bipa_vpr.stg_calendario",
     "create table proceso_bipa_vpr.stg_calendario as select * from s_core.calendario;"),
    ("s_core.oficiales", "proceso_bipa_vpr.stg_oficiales",
     "create table proceso_bipa_vpr.stg_oficiales as select * from s_core.oficiales;"),
    ("s_core.segmentos", "proceso_bipa_vpr.stg_segmentos",
     "create table proceso_bipa_vpr.stg_segmentos as select * from s_core.segmentos;"),
    ("s_core.monedas", "proceso_bipa_vpr.stg_monedas",
     "create table proceso_bipa_vpr.stg_monedas as select * from s_core.monedas;"),
    ("s_core.tipo_cambio", "proceso_bipa_vpr.stg_tipo_cambio",
     "create table proceso_bipa_vpr.stg_tipo_cambio as select * from s_core.tipo_cambio;"),
    ("s_core.cobros", "proceso_bipa_vpr.stg_cobros",
     "create table proceso_bipa_vpr.stg_cobros as select * from s_core.cobros;"),
    ("s_core.mora_hist", "proceso_riesgo.stg_mora_hist",
     "create table proceso_riesgo.stg_mora_hist as select * from s_core.mora_hist;"),
    ("s_core.score_externo", "proceso_riesgo.stg_score_externo",
     "create table proceso_riesgo.stg_score_externo as select * from s_core.score_externo;"),
    ("s_core.ingresos", "proceso_bipa_vpr.stg_ingresos",
     "create table proceso_bipa_vpr.stg_ingresos as select * from s_core.ingresos;"),
    ("s_core.egresos", "proceso_bipa_vpr.stg_egresos",
     "create table proceso_bipa_vpr.stg_egresos as select * from s_core.egresos;"),
    ("s_core.localizacion", "proceso_bipa_vpr.stg_localizacion",
     "create table proceso_bipa_vpr.stg_localizacion as select * from s_core.localizacion;"),

    # =========================
    # Staging -> tablas proceso base
    # =========================
    ("proceso_bipa_vpr.stg_clientes", "proceso_bipa_vpr.dim_clientes",
     "create table proceso_bipa_vpr.dim_clientes as select ... from proceso_bipa_vpr.stg_clientes;"),
    ("proceso_bipa_vpr.stg_segmentos", "proceso_bipa_vpr.dim_clientes",
     "create table proceso_bipa_vpr.dim_clientes as select ... from proceso_bipa_vpr.stg_clientes c left join proceso_bipa_vpr.stg_segmentos s on ...;"),
    ("proceso_bipa_vpr.stg_localizacion", "proceso_bipa_vpr.dim_clientes",
     "create table proceso_bipa_vpr.dim_clientes as select ... from proceso_bipa_vpr.stg_clientes c left join proceso_bipa_vpr.stg_localizacion l on ...;"),

    ("proceso_bipa_vpr.stg_productos", "proceso_bipa_vpr.dim_productos",
     "create table proceso_bipa_vpr.dim_productos as select ... from proceso_bipa_vpr.stg_productos;"),
    ("proceso_bipa_vpr.stg_tasas", "proceso_bipa_vpr.dim_productos",
     "create table proceso_bipa_vpr.dim_productos as select ... from proceso_bipa_vpr.stg_productos p left join proceso_bipa_vpr.stg_tasas t on ...;"),

    ("proceso_bipa_vpr.stg_sucursales", "proceso_bipa_vpr.dim_sucursales",
     "create table proceso_bipa_vpr.dim_sucursales as select ... from proceso_bipa_vpr.stg_sucursales;"),
    ("proceso_bipa_vpr.stg_oficiales", "proceso_bipa_vpr.dim_oficiales",
     "create table proceso_bipa_vpr.dim_oficiales as select ... from proceso_bipa_vpr.stg_oficiales;"),
    ("proceso_bipa_vpr.stg_calendario", "proceso_bipa_vpr.dim_tiempo",
     "create table proceso_bipa_vpr.dim_tiempo as select ... from proceso_bipa_vpr.stg_calendario;"),
    ("proceso_bipa_vpr.stg_monedas", "proceso_bipa_vpr.dim_monedas",
     "create table proceso_bipa_vpr.dim_monedas as select ... from proceso_bipa_vpr.stg_monedas;"),
    ("proceso_bipa_vpr.stg_tipo_cambio", "proceso_bipa_vpr.dim_tipo_cambio",
     "create table proceso_bipa_vpr.dim_tipo_cambio as select ... from proceso_bipa_vpr.stg_tipo_cambio;"),

    ("proceso_bipa_vpr.stg_prestamos", "proceso_bipa_vpr.fact_prestamos_stg",
     "create table proceso_bipa_vpr.fact_prestamos_stg as select ... from proceso_bipa_vpr.stg_prestamos;"),
    ("proceso_bipa_vpr.stg_garantias", "proceso_bipa_vpr.fact_prestamos_stg",
     "create table proceso_bipa_vpr.fact_prestamos_stg as select ... from proceso_bipa_vpr.stg_prestamos p left join proceso_bipa_vpr.stg_garantias g on ...;"),
    ("proceso_bipa_vpr.stg_tasas", "proceso_bipa_vpr.fact_prestamos_stg",
     "create table proceso_bipa_vpr.fact_prestamos_stg as select ... from proceso_bipa_vpr.stg_prestamos p left join proceso_bipa_vpr.stg_tasas t on ...;"),

    ("proceso_bipa_vpr.stg_pagos", "proceso_bipa_vpr.fact_pagos_stg",
     "create table proceso_bipa_vpr.fact_pagos_stg as select ... from proceso_bipa_vpr.stg_pagos;"),
    ("proceso_bipa_vpr.stg_cobros", "proceso_bipa_vpr.fact_pagos_stg",
     "create table proceso_bipa_vpr.fact_pagos_stg as select ... from proceso_bipa_vpr.stg_pagos p left join proceso_bipa_vpr.stg_cobros c on ...;"),

    ("proceso_bipa_vpr.stg_transacciones", "proceso_bipa_vpr.fact_transacciones_stg",
     "create table proceso_bipa_vpr.fact_transacciones_stg as select ... from proceso_bipa_vpr.stg_transacciones;"),
    ("proceso_bipa_vpr.stg_ingresos", "proceso_bipa_vpr.fact_ingresos_stg",
     "create table proceso_bipa_vpr.fact_ingresos_stg as select ... from proceso_bipa_vpr.stg_ingresos;"),
    ("proceso_bipa_vpr.stg_egresos", "proceso_bipa_vpr.fact_egresos_stg",
     "create table proceso_bipa_vpr.fact_egresos_stg as select ... from proceso_bipa_vpr.stg_egresos;"),

    ("proceso_riesgo.stg_cartera", "proceso_riesgo.mora_clientes",
     "create table proceso_riesgo.mora_clientes as select ... from proceso_riesgo.stg_cartera;"),
    ("proceso_riesgo.stg_mora_hist", "proceso_riesgo.mora_clientes",
     "create table proceso_riesgo.mora_clientes as select ... from proceso_riesgo.stg_cartera c left join proceso_riesgo.stg_mora_hist m on ...;"),

    ("proceso_riesgo.stg_score_externo", "proceso_riesgo.score_enriquecido",
     "create table proceso_riesgo.score_enriquecido as select ... from proceso_riesgo.stg_score_externo;"),
    ("proceso_bipa_vpr.stg_clientes", "proceso_riesgo.score_enriquecido",
     "create table proceso_riesgo.score_enriquecido as select ... from proceso_riesgo.stg_score_externo s left join proceso_bipa_vpr.stg_clientes c on ...;"),

    # =========================
    # Proceso -> proceso intermedio (más profundidad)
    # =========================
    ("proceso_bipa_vpr.fact_prestamos_stg", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg;"),
    ("proceso_bipa_vpr.dim_clientes", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_productos", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_productos dp on ...;"),
    ("proceso_bipa_vpr.dim_sucursales", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_sucursales ds on ...;"),
    ("proceso_bipa_vpr.dim_oficiales", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_oficiales dof on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "proceso_bipa_vpr.fact_prestamos_enriquecido",
     "create table proceso_bipa_vpr.fact_prestamos_enriquecido as select ... from proceso_bipa_vpr.fact_prestamos_stg fp join proceso_bipa_vpr.dim_tiempo dt on ...;"),

    ("proceso_bipa_vpr.fact_pagos_stg", "proceso_bipa_vpr.fact_pagos_enriquecido",
     "create table proceso_bipa_vpr.fact_pagos_enriquecido as select ... from proceso_bipa_vpr.fact_pagos_stg;"),
    ("proceso_bipa_vpr.dim_clientes", "proceso_bipa_vpr.fact_pagos_enriquecido",
     "create table proceso_bipa_vpr.fact_pagos_enriquecido as select ... from proceso_bipa_vpr.fact_pagos_stg fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "proceso_bipa_vpr.fact_pagos_enriquecido",
     "create table proceso_bipa_vpr.fact_pagos_enriquecido as select ... from proceso_bipa_vpr.fact_pagos_stg fp join proceso_bipa_vpr.dim_tiempo dt on ...;"),
    ("proceso_bipa_vpr.dim_monedas", "proceso_bipa_vpr.fact_pagos_enriquecido",
     "create table proceso_bipa_vpr.fact_pagos_enriquecido as select ... from proceso_bipa_vpr.fact_pagos_stg fp join proceso_bipa_vpr.dim_monedas dm on ...;"),

    ("proceso_bipa_vpr.fact_transacciones_stg", "proceso_bipa_vpr.fact_transacciones_enriquecido",
     "create table proceso_bipa_vpr.fact_transacciones_enriquecido as select ... from proceso_bipa_vpr.fact_transacciones_stg;"),
    ("proceso_bipa_vpr.dim_clientes", "proceso_bipa_vpr.fact_transacciones_enriquecido",
     "create table proceso_bipa_vpr.fact_transacciones_enriquecido as select ... from proceso_bipa_vpr.fact_transacciones_stg ft join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "proceso_bipa_vpr.fact_transacciones_enriquecido",
     "create table proceso_bipa_vpr.fact_transacciones_enriquecido as select ... from proceso_bipa_vpr.fact_transacciones_stg ft join proceso_bipa_vpr.dim_tiempo dt on ...;"),
    ("proceso_bipa_vpr.dim_sucursales", "proceso_bipa_vpr.fact_transacciones_enriquecido",
     "create table proceso_bipa_vpr.fact_transacciones_enriquecido as select ... from proceso_bipa_vpr.fact_transacciones_stg ft join proceso_bipa_vpr.dim_sucursales ds on ...;"),

    ("proceso_riesgo.mora_clientes", "proceso_riesgo.mora_enriquecida",
     "create table proceso_riesgo.mora_enriquecida as select ... from proceso_riesgo.mora_clientes;"),
    ("proceso_riesgo.score_enriquecido", "proceso_riesgo.mora_enriquecida",
     "create table proceso_riesgo.mora_enriquecida as select ... from proceso_riesgo.mora_clientes m left join proceso_riesgo.score_enriquecido s on ...;"),
    ("proceso_bipa_vpr.dim_clientes", "proceso_riesgo.mora_enriquecida",
     "create table proceso_riesgo.mora_enriquecida as select ... from proceso_riesgo.mora_clientes m left join proceso_bipa_vpr.dim_clientes dc on ...;"),

    # =========================
    # Proceso -> resultados (facts y alertas)
    # =========================
    ("proceso_bipa_vpr.fact_prestamos_enriquecido", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_enriquecido;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_enriquecido fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_productos", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_enriquecido fp join proceso_bipa_vpr.dim_productos dp on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "resultados_bipa_vpr.tb_fact_prestamos",
     "create table resultados_bipa_vpr.tb_fact_prestamos as select ... from proceso_bipa_vpr.fact_prestamos_enriquecido fp join proceso_bipa_vpr.dim_tiempo dt on ...;"),

    ("proceso_bipa_vpr.fact_pagos_enriquecido", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_enriquecido;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_enriquecido fp join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "resultados_bipa_vpr.tb_fact_pagos",
     "create table resultados_bipa_vpr.tb_fact_pagos as select ... from proceso_bipa_vpr.fact_pagos_enriquecido fp join proceso_bipa_vpr.dim_tiempo dt on ...;"),

    ("proceso_bipa_vpr.fact_transacciones_enriquecido", "resultados_bipa_vpr.tb_fact_transacciones",
     "create table resultados_bipa_vpr.tb_fact_transacciones as select ... from proceso_bipa_vpr.fact_transacciones_enriquecido;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_bipa_vpr.tb_fact_transacciones",
     "create table resultados_bipa_vpr.tb_fact_transacciones as select ... from proceso_bipa_vpr.fact_transacciones_enriquecido ft join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "resultados_bipa_vpr.tb_fact_transacciones",
     "create table resultados_bipa_vpr.tb_fact_transacciones as select ... from proceso_bipa_vpr.fact_transacciones_enriquecido ft join proceso_bipa_vpr.dim_tiempo dt on ...;"),

    ("proceso_riesgo.mora_enriquecida", "resultados_riesgo.tb_alertas_mora",
     "create table resultados_riesgo.tb_alertas_mora as select ... from proceso_riesgo.mora_enriquecida;"),
    ("proceso_riesgo.score_enriquecido", "resultados_riesgo.tb_alertas_mora",
     "create table resultados_riesgo.tb_alertas_mora as select ... from proceso_riesgo.mora_enriquecida m left join proceso_riesgo.score_enriquecido s on ...;"),

    ("proceso_riesgo.mora_enriquecida", "resultados_riesgo.tb_score_riesgo",
     "create table resultados_riesgo.tb_score_riesgo as select ... from proceso_riesgo.mora_enriquecida;"),
    ("proceso_bipa_vpr.dim_clientes", "resultados_riesgo.tb_score_riesgo",
     "create table resultados_riesgo.tb_score_riesgo as select ... from proceso_riesgo.mora_enriquecida m join proceso_bipa_vpr.dim_clientes dc on ...;"),
    ("proceso_bipa_vpr.dim_tiempo", "resultados_riesgo.tb_score_riesgo",
     "create table resultados_riesgo.tb_score_riesgo as select ... from proceso_riesgo.mora_enriquecida m join proceso_bipa_vpr.dim_tiempo dt on ...;"),

    # =========================
    # Resultados -> resultados (reportes)
    # =========================
    ("resultados_bipa_vpr.tb_fact_prestamos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_prestamos;"),
    ("resultados_bipa_vpr.tb_fact_pagos", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_pagos;"),
    ("resultados_bipa_vpr.tb_fact_transacciones", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_bipa_vpr.tb_fact_transacciones;"),
    ("resultados_riesgo.tb_alertas_mora", "resultados_bipa_vpr.tb_reporte_mensual",
     "create table resultados_bipa_vpr.tb_reporte_mensual as select ... from resultados_riesgo.tb_alertas_mora;"),

    ("resultados_bipa_vpr.tb_fact_prestamos", "resultados_bipa_vpr.tb_reporte_cartera",
     "create table resultados_bipa_vpr.tb_reporte_cartera as select ... from resultados_bipa_vpr.tb_fact_prestamos;"),
    ("resultados_riesgo.tb_score_riesgo", "resultados_bipa_vpr.tb_reporte_cartera",
     "create table resultados_bipa_vpr.tb_reporte_cartera as select ... from resultados_riesgo.tb_score_riesgo;"),

    ("resultados_bipa_vpr.tb_reporte_mensual", "resultados_bipa_vpr.tb_dashboard_ejecutivo",
     "create table resultados_bipa_vpr.tb_dashboard_ejecutivo as select ... from resultados_bipa_vpr.tb_reporte_mensual;"),
    ("resultados_bipa_vpr.tb_reporte_cartera", "resultados_bipa_vpr.tb_dashboard_ejecutivo",
     "create table resultados_bipa_vpr.tb_dashboard_ejecutivo as select ... from resultados_bipa_vpr.tb_reporte_cartera;"),
    ("resultados_riesgo.tb_alertas_mora", "resultados_bipa_vpr.tb_dashboard_ejecutivo",
     "create table resultados_bipa_vpr.tb_dashboard_ejecutivo as select ... from resultados_riesgo.tb_alertas_mora;"),
    ("resultados_riesgo.tb_score_riesgo", "resultados_bipa_vpr.tb_dashboard_ejecutivo",
     "create table resultados_bipa_vpr.tb_dashboard_ejecutivo as select ... from resultados_riesgo.tb_score_riesgo;"),

    # =========================
    # Relaciones extra (complejidad)
    # =========================
    ("proceso_bipa_vpr.stg_clientes", "proceso_riesgo.score_enriquecido",
     "insert overwrite table proceso_riesgo.score_enriquecido select ... from proceso_riesgo.stg_score_externo se join proceso_bipa_vpr.stg_clientes c on ...;"),
    ("proceso_bipa_vpr.fact_pagos_stg", "proceso_riesgo.mora_enriquecida",
     "insert overwrite table proceso_riesgo.mora_enriquecida select ... from proceso_riesgo.mora_clientes m left join proceso_bipa_vpr.fact_pagos_stg p on ...;"),
    ("proceso_bipa_vpr.fact_prestamos_enriquecido", "resultados_riesgo.tb_score_riesgo",
     "insert into resultados_riesgo.tb_score_riesgo select ... from proceso_bipa_vpr.fact_prestamos_enriquecido fp;"),
    ("resultados_bipa_vpr.tb_fact_pagos", "resultados_bipa_vpr.tb_reporte_cartera",
     "insert into resultados_bipa_vpr.tb_reporte_cartera select ... from resultados_bipa_vpr.tb_fact_pagos;"),
    ("resultados_bipa_vpr.tb_fact_transacciones", "resultados_bipa_vpr.tb_reporte_cartera",
     "insert into resultados_bipa_vpr.tb_reporte_cartera select ... from resultados_bipa_vpr.tb_fact_transacciones;"),

    # =========================
    # Self-loops (para probar lógica de visitados/duplicados)
    # =========================
    ("proceso_bipa_vpr.fact_pagos_stg", "proceso_bipa_vpr.fact_pagos_stg",
     "insert overwrite table proceso_bipa_vpr.fact_pagos_stg select ... from proceso_bipa_vpr.fact_pagos_stg;"),
    ("resultados_bipa_vpr.tb_reporte_mensual", "resultados_bipa_vpr.tb_reporte_mensual",
     "insert overwrite table resultados_bipa_vpr.tb_reporte_mensual select ... from resultados_bipa_vpr.tb_reporte_mensual;"),
]
insumo_base = pd.DataFrame(insumo_rows, columns=["tabla_origen","tabla_destino","query"])
# -----------------------------
# Utilidades
# -----------------------------
def parse_schema(fqn: str):
    fqn = (fqn or "").strip().lower()
    return fqn.split(".", 1)[0] if "." in fqn else fqn

def zone_prefix(fqn: str):
    schema = parse_schema(fqn)
    if schema.startswith("s_"):
        return "S"
    if schema.startswith("proceso"):
        return "P"
    if schema.startswith("resultados") or schema.startswith("resultado"):
        return "R"
    return "U"

def zone_color(fqn: str):
    z = zone_prefix(fqn)
    if z == "S":
        return "#e74c3c"
    if z == "P":
        return "#f1c40f"
    if z == "R":
        return "#2ecc71"
    return "#95a5a6"

def svg_escape(s: str):
    return (str(s)
            .replace("&","&amp;")
            .replace("<","&lt;")
            .replace(">","&gt;")
            .replace('"',"&quot;"))

# -----------------------------
# Construir índices
# -----------------------------
incoming = defaultdict(list)  # dst -> [(src, query)]
outgoing = defaultdict(list)  # src -> [(dst, query)]
for _, r in insumo_base.iterrows():
    s = r["tabla_origen"].strip()
    d = r["tabla_destino"].strip()
    incoming[d].append((s, r["query"]))
    outgoing[s].append((d, r["query"]))


# -----------------------------
# Closures
# -----------------------------
def upstream_closure(target: str):
    """
    Linaje upstream SIN duplicados.
    - visited_nodes evita re-procesar nodos (ciclos)
    - seen_edges evita repetir relaciones
    - si hay self-loop (A->A) se incluye 1 vez
    """
    target = target.strip()
    visited_nodes = set([target])
    q = deque([target])

    seen_edges = set()   # (src, dst, query)
    edges = []

    while q:
        d = q.popleft()

        for s, qry in incoming.get(d, []):
            s = (s or "").strip()
            edge_key = (s, d, (qry or "").strip())

            if edge_key not in seen_edges:
                seen_edges.add(edge_key)
                edges.append((s, d, qry))

            # Encolar sólo si el nodo no fue visitado (evita ciclos)
            if s not in visited_nodes:
                visited_nodes.add(s)
                q.append(s)

    return visited_nodes, edges


def downstream_closure(root: str):
    """
    Linaje downstream SIN duplicados.
    - visited_nodes evita re-procesar nodos (ciclos)
    - seen_edges evita repetir relaciones
    - si hay self-loop (A->A) se incluye 1 vez
    """
    root = root.strip()
    visited_nodes = set([root])
    q = deque([root])

    seen_edges = set()   # (src, dst, query)
    edges = []

    while q:
        s = q.popleft()

        for d, qry in outgoing.get(s, []):
            d = (d or "").strip()
            edge_key = (s, d, (qry or "").strip())

            if edge_key not in seen_edges:
                seen_edges.add(edge_key)
                edges.append((s, d, qry))

            if d not in visited_nodes:
                visited_nodes.add(d)
                q.append(d)

    return visited_nodes, edges

# -----------------------------
# Levels para layout
# -----------------------------

def levels_upstream(edges, target):
    """
    Niveles para layout upstream (target a la derecha).
    Calcula distancia mínima en hops desde cada nodo hacia target (en grafo invertido).
    Ignora self-loops para layout.
    """
    target = (target or "").strip()

    # d -> [s]  (aristas src->dst, para upstream caminamos dst -> src)
    rev_adj = defaultdict(list)
    for s, d, _ in edges:
        s = (s or "").strip()
        d = (d or "").strip()
        if not s or not d:
            continue
        if s == d:
            continue  # self-loop no afecta layout
        rev_adj[d].append(s)

    level = {target: 0}
    q = deque([target])

    while q:
        cur = q.popleft()
        for parent in rev_adj.get(cur, []):
            if parent not in level:
                level[parent] = level[cur] + 1
                q.append(parent)

    return level


def levels_downstream(edges, root):
    """
    Niveles para layout downstream (root a la izquierda).
    Calcula distancia mínima en hops desde root hacia cada nodo.
    Ignora self-loops para layout.
    """
    root = (root or "").strip()

    # s -> [d]
    adj = defaultdict(list)
    for s, d, _ in edges:
        s = (s or "").strip()
        d = (d or "").strip()
        if not s or not d:
            continue
        if s == d:
            continue  # self-loop no afecta layout
        adj[s].append(d)

    level = {root: 0}
    q = deque([root])

    while q:
        cur = q.popleft()
        for nxt in adj.get(cur, []):
            if nxt not in level:
                level[nxt] = level[cur] + 1
                q.append(nxt)

    return level

# -----------------------------
# IDs (reinicio por objetivo)
# -----------------------------
def assign_ids_per_objective(nodes, level_map, kind):
    def sort_key(n):
        lv = level_map.get(n, -1)
        return ((-lv, n.lower()) if kind == "upstream" else (lv, n.lower()))
    ordered = sorted(nodes, key=sort_key)

    counters = {"S":1, "P":1, "R":1, "U":1}
    node_id = {}
    for n in ordered:
        z = zone_prefix(n)
        node_id[n] = f"{z}{counters[z]}"
        counters[z] += 1
    return node_id


def build_canvas_html(nodes, edges, level_map, kind, node_ids, uid="g1", viewport_h=320, objetivo_fqn=""):
    dx = 400
    dy = 150
    node_w, node_h = 270, 74

    max_level = max(level_map.values()) if level_map else 0

    if kind == "upstream":
        x_level = {n: (max_level - level_map.get(n, max_level)) * dx for n in nodes}
    else:
        x_level = {n: level_map.get(n, 0) * dx for n in nodes}

    buckets = defaultdict(list)
    for n in nodes:
        buckets[level_map.get(n, 0)].append(n)
    for k in buckets:
        buckets[k].sort(key=lambda n: n.lower())

    y_pos = {}
    for k, ns in buckets.items():
        for i, n in enumerate(ns):
            y_pos[n] = 60 + i * dy

    base_w = (max_level + 1) * dx + 520
    base_h = (max(y_pos.values()) if y_pos else 0) + 260

    toolbar_h = 38

    # Nodos para JS (incluye name/id/rect/color)
    nodes_list = []
    for n in nodes:
        nodes_list.append({
            "name": n,
            "id": node_ids.get(n, ""),
            "x": float(x_level[n]),
            "y": float(y_pos[n]),
            "w": float(node_w),
            "h": float(node_h),
            "color": zone_color(n)
        })

    # Edges para JS (incluye src/dst por nombre y coords)
    # Edges para JS (incluye src/dst por nombre y coords)
    edges_list = []
    for s, d, _ in edges:
        if s not in x_level or d not in x_level:
            continue

        is_self = (s.strip().lower() == d.strip().lower())

        edges_list.append({
            "src": s,
            "dst": d,
            "self_loop": bool(is_self),

            # coords base (para enlaces normales)
            "sx": float(x_level[s] + node_w),
            "sy": float(y_pos[s] + node_h/2),
            "tx": float(x_level[d]),
            "ty": float(y_pos[d] + node_h/2),

            # rect del nodo destino (útil para dibujar self-loop local)
            "dx": float(x_level[d]),
            "dy": float(y_pos[d]),
            "dw": float(node_w),
            "dh": float(node_h)
        })

    nodes_json = json.dumps(nodes_list)
    edges_json = json.dumps(edges_list)
    objetivo_json = json.dumps(objetivo_fqn)

    html = f"""
<div id="wrap_{uid}" style="position:relative; width:100%; height:{viewport_h}px; overflow:hidden; background:#ffffff; font-family:Segoe UI, Arial;">
  <div style="display:flex; gap:8px; align-items:center; padding:6px 8px; height:{toolbar_h}px; box-sizing:border-box; border-bottom:1px solid #ddd;">
    <button id="zin_{uid}" style="padding:4px 10px;">zoom in</button>
    <button id="zout_{uid}" style="padding:4px 10px;">zoom out</button>
    <button id="zreset_{uid}" style="padding:4px 10px;">Reset</button>
    <span id="zlbl_{uid}" style="margin-left:8px; color:#444; font-size:12px;">100%</span>
    <span style="margin-left:10px; color:#777; font-size:11px;">(Ctrl + rueda para zoom)</span>
  </div>

  <div id="vp_{uid}" style="height:calc(100% - {toolbar_h}px); overflow:auto; background:#fff;">
    <div id="world_{uid}" data-basew="{base_w}" data-baseh="{base_h}"
         style="width:{base_w}px; height:{base_h}px; position:relative;">
      <canvas id="cv_{uid}" width="{base_w}" height="{base_h}"
              style="display:block; background:#ffffff;"></canvas>
    </div>
  </div>

  <!-- POPUP -->
  <div id="info_{uid}"
       style="position:absolute; top:{toolbar_h + 8}px; right:10px;
              width:360px; max-height:60%;
              overflow:auto;
              background:#fff; border:1px solid #bbb; border-radius:8px;
              box-shadow:0 6px 20px rgba(0,0,0,.12);
              padding:10px 12px;
              display:none;
              z-index:9999;">
    <div style="font-weight:700; margin-bottom:6px;">Detalle del nodo</div>
    <div style="font-size:12px; color:#555; margin-bottom:8px;">
      <div><b>Tabla objetivo:</b> <span id="obj_{uid}"></span></div>
      <div style="margin-top:4px;"><b>Nodo seleccionado:</b> <span id="sel_{uid}"></span></div>
    </div>
    <div style="font-weight:700; margin:8px 0 6px 0;">Tablas origen directas</div>
    <ul id="orig_{uid}" style="margin:0; padding-left:18px; font-size:12px;"></ul>
    <div style="margin-top:8px; font-size:11px; color:#777;">
      * Los enlaces origen a nodo se resaltan en <span style="color:#c0392b;font-weight:700;">rojo</span>.
    </div>
  </div>
</div>

<script>
(function() {{
  var nodes = {nodes_json};
  var edges = {edges_json};
  var objetivo = {objetivo_json};

  var scale = 1.0;
  var minS = 0.05, maxS = 3.0;

  var vp = document.getElementById("vp_{uid}");
  var world = document.getElementById("world_{uid}");
  var canvas = document.getElementById("cv_{uid}");
  var ctx = canvas.getContext("2d");
  var lbl = document.getElementById("zlbl_{uid}");

  var info = document.getElementById("info_{uid}");
  var objSpan = document.getElementById("obj_{uid}");
  var selSpan = document.getElementById("sel_{uid}");
  var origUl = document.getElementById("orig_{uid}");

  // Estado de selección
  var selectedName = null;
  var selectedOrigins = new Set(); // src directas del seleccionado

  var baseW = parseFloat(world.getAttribute("data-basew")) || 1000;
  var baseH = parseFloat(world.getAttribute("data-baseh")) || 600;

  function clamp(v) {{ return Math.max(minS, Math.min(maxS, v)); }}

  function setWorldSize() {{
    var w = Math.round(baseW * scale);
    var h = Math.round(baseH * scale);
    world.style.width = w + "px";
    world.style.height = h + "px";
    canvas.width = w;
    canvas.height = h;
    if(lbl) lbl.textContent = Math.round(scale * 100) + "%";
  }}

  function drawSelfLoopForNode(n, color, lineW, isHighlighted) {{
    // n viene en coordenadas "mundo" (sin escala)
    var x = n.dx * scale;
    var y = n.dy * scale;
    var w = n.dw * scale;
    var h = n.dh * scale;

    // Punto de entrada arriba del nodo (centro superior)
    var entryX = x + w * 0.55;
    var entryY = y;

    // Subida del loop y salida a la izquierda (como tu dibujo)
    var topY = y - 42 * scale;
    var leftX = x + 18 * scale;

    ctx.beginPath();
    ctx.moveTo(entryX, entryY);

    // flecha baja hacia el nodo (segmento corto vertical)
    ctx.lineTo(entryX, topY + 18 * scale);

    // recorrido loop: arriba -> izquierda -> abajo
    ctx.lineTo(entryX, topY);
    ctx.lineTo(leftX, topY);
    ctx.lineTo(leftX, y + h * 0.55);

    ctx.strokeStyle = color;
    ctx.lineWidth = lineW;
    ctx.stroke();

    // punta de flecha hacia abajo entrando al nodo
    var ah = 8 * scale;  // arrow size
    var aw = 5 * scale;
    ctx.beginPath();
    ctx.moveTo(entryX, entryY);
    ctx.lineTo(entryX - aw, entryY - ah);
    ctx.lineTo(entryX + aw, entryY - ah);
    ctx.closePath();
    ctx.fillStyle = color;
    ctx.fill();
  }}

  function drawRoundedRect(x,y,w,h,r) {{
    ctx.beginPath();
    ctx.moveTo(x+r, y);
    ctx.lineTo(x+w-r, y);
    ctx.quadraticCurveTo(x+w, y, x+w, y+r);
    ctx.lineTo(x+w, y+h-r);
    ctx.quadraticCurveTo(x+w, y+h, x+w-r, y+h);
    ctx.lineTo(x+r, y+h);
    ctx.quadraticCurveTo(x, y+h, x, y+h-r);
    ctx.lineTo(x, y+r);
    ctx.quadraticCurveTo(x, y, x+r, y);
    ctx.closePath();
  }}

  function drawArrow(x1,y1,x2,y2, color, width) {{
    var mx = (x1 + x2) / 2;
    ctx.strokeStyle = color;
    ctx.lineWidth = width;
    ctx.beginPath();
    ctx.moveTo(x1, y1);
    ctx.bezierCurveTo(mx, y1, mx, y2, x2, y2);
    ctx.stroke();

    // Flecha (triángulo)
    var angle = Math.atan2(y2 - y1, x2 - x1);
    var headLen = 10 * (width/2);
    var a1 = angle + Math.PI/7;
    var a2 = angle - Math.PI/7;

    ctx.fillStyle = color;
    ctx.beginPath();
    ctx.moveTo(x2, y2);
    ctx.lineTo(x2 - headLen*Math.cos(a1), y2 - headLen*Math.sin(a1));
    ctx.lineTo(x2 - headLen*Math.cos(a2), y2 - headLen*Math.sin(a2));
    ctx.closePath();
    ctx.fill();
  }}

  function wrapText(text, maxChars) {{
    var out = [];
    var s = (text || "");
    while(s.length > maxChars) {{
      out.push(s.slice(0, maxChars));
      s = s.slice(maxChars);
    }}
    if(s.length) out.push(s);
    return out;
  }}

  function render() {{
    ctx.clearRect(0,0,canvas.width,canvas.height);

    // Edges primero: si están seleccionadas, rojo
    // Edges primero: si están seleccionadas, rojo

    for (var i = 0; i < edges.length; i++) {{
      var e = edges[i];

      var isRed = (selectedName && e.dst === selectedName && selectedOrigins.has(e.src));
      var color = isRed ? "#c0392b" : "#555";
      var wLine = isRed ? 3 : 2;

      // Self-loop: dibujar localmente sobre el nodo, NO como flecha normal
      if (e.self_loop) {{
        drawSelfLoopForNode(e, color, wLine, isRed);
        continue;
      }}

      // Enlace normal
      drawArrow(e.sx * scale, e.sy * scale, e.tx * scale, e.ty * scale, color, wLine);
    }}

    // Nodes
    for(var j=0;j<nodes.length;j++) {{
      var n = nodes[j];
      var x = n.x*scale, y = n.y*scale, w = n.w*scale, h = n.h*scale;

      ctx.fillStyle = n.color;
      ctx.strokeStyle = "#333";
      ctx.lineWidth = 1.2;
      drawRoundedRect(x,y,w,h,10*scale);
      ctx.fill();
      ctx.stroke();

      // Texto
      ctx.fillStyle = "#111";
      ctx.font = (14*scale).toFixed(0) + "px Segoe UI, Arial";
      ctx.fillText(n.id, x + 12*scale, y + 28*scale);

      ctx.font = (12*scale).toFixed(0) + "px Segoe UI, Arial";
      var lines = wrapText(n.name, 34);
      for(var k=0;k<Math.min(2, lines.length);k++) {{
        ctx.fillText(lines[k], x + 12*scale, y + (50 + k*16)*scale);
      }}
    }}
  }}

  function apply() {{
    setWorldSize();
    render();
  }}

  function zoomAt(mult, clientX, clientY) {{
    if(!vp) return;
    var rect = vp.getBoundingClientRect();
    var x = clientX - rect.left;
    var y = clientY - rect.top;

    var prev = scale;
    var scrollX = vp.scrollLeft + x;
    var scrollY = vp.scrollTop  + y;

    scale = clamp(scale * mult);
    apply();

    var ratio = scale / prev;
    vp.scrollLeft = Math.round(scrollX * ratio - x);
    vp.scrollTop  = Math.round(scrollY * ratio - y);
  }}

  function openPopup(nodeName) {{
    selectedName = nodeName;
    selectedOrigins = new Set();

    // Orígenes directos: edges src -> selectedName
    for(var i=0;i<edges.length;i++) {{
      if(edges[i].dst === selectedName) selectedOrigins.add(edges[i].src);
    }}

    // Pintar popup
    if(objSpan) objSpan.textContent = objetivo || "";
    if(selSpan) selSpan.textContent = selectedName || "";

    if(origUl) {{
      origUl.innerHTML = "";
      var arr = Array.from(selectedOrigins).sort();
      if(arr.length === 0) {{
        var li = document.createElement("li");
        li.textContent = "(sin origenes directos en este grafo)";
        origUl.appendChild(li);
      }} else {{
        for(var k=0;k<arr.length;k++) {{
          var li = document.createElement("li");
          li.textContent = arr[k];
          origUl.appendChild(li);
        }}
      }}
    }}

    if(info) info.style.display = "block";
    render(); // para resaltar enlaces
  }}

  function closePopup() {{
    selectedName = null;
    selectedOrigins = new Set();
    if(info) info.style.display = "none";
    render(); // quitar resaltado
  }}

  // Hit-test: click sobre nodo
  function findNodeAt(px, py) {{
    // px, py vienen en coordenadas CANVAS (ya escaladas), pasamos a mundo
    var mx = px / scale;
    var my = py / scale;

    // iterar al revés para priorizar el “último dibujado”
    for(var i=nodes.length-1; i>=0; i--) {{
      var n = nodes[i];
      if(mx >= n.x && mx <= n.x + n.w && my >= n.y && my <= n.y + n.h) {{
        return n.name;
      }}
    }}
    return null;
  }}

  // Clicks
  canvas.addEventListener("click", function(e) {{
    var name = findNodeAt(e.offsetX, e.offsetY);
    if(name) {{
      // Selecciona nodo (cierra anterior y abre el nuevo automáticamente)
      openPopup(name);
    }} else {{
      // Click fuera de nodos: cerrar
      closePopup();
    }}
  }});

  // Click dentro del popup NO lo cierra
  info.addEventListener("click", function(e) {{
    e.stopPropagation();
  }});

  // Click fuera del popup (pero dentro del contenedor) lo cierra
  document.getElementById("wrap_{uid}").addEventListener("click", function(e) {{
    // si el click fue en el canvas, ya se manejó arriba
    // si fue en otra zona y el popup está abierto, cierra
    if(info && info.style.display === "block") {{
      // si el click no fue dentro del popup
      if(!info.contains(e.target) && e.target !== canvas) {{
        closePopup();
      }}
    }}
  }});

  // Botones
  var zin = document.getElementById("zin_{uid}");
  var zout = document.getElementById("zout_{uid}");
  var zreset = document.getElementById("zreset_{uid}");

  if(zin) zin.addEventListener("click", function() {{
    var r = vp.getBoundingClientRect();
    zoomAt(1.1, r.left + 30, r.top + 30);
  }});
  if(zout) zout.addEventListener("click", function() {{
    var r = vp.getBoundingClientRect();
    zoomAt(0.9, r.left + 30, r.top + 30);
  }});
  if(zreset) zreset.addEventListener("click", function() {{
    scale = 1.0; apply();
  }});

  // Ctrl + rueda
  vp.addEventListener("wheel", function(e) {{
    if(!(e.ctrlKey || e.metaKey)) return;
    e.preventDefault();
    var ds = (e.deltaY < 0) ? 1.1 : 0.9;
    zoomAt(ds, e.clientX, e.clientY);
  }}, {{ passive:false }});

  apply();
}})();
</script>
""".strip()

    return html


# -----------------------------
# Objetivos:
#  A) todos los tabla_destino (upstream)
#  B) todas las tablas crudas s_* que aparezcan como tabla_origen (downstream)
# -----------------------------
dest_targets = sorted(insumo_base["tabla_destino"].unique(), key=lambda s: s.lower())
raw_sources = sorted({t for t in insumo_base["tabla_origen"].unique() if parse_schema(t).startswith("s_")}, key=lambda s: s.lower())

objectives = [("upstream", t) for t in dest_targets] + [("downstream", s) for s in raw_sources]

# -----------------------------
# Generar tb_linaje y tb_html
# -----------------------------
tb_linaje_rows = []
tb_html_rows = []

for kind, obj in objectives:
    if kind == "upstream":
        nodes, edges = upstream_closure(obj)
        level_map = levels_upstream(edges, obj)
    else:
        nodes, edges = downstream_closure(obj)
        level_map = levels_downstream(edges, obj)

    node_ids = assign_ids_per_objective(nodes, level_map, kind)

    uid = str(abs(hash(obj)) % 10_000_000)
    #html = build_svg_html(nodes, edges, level_map, kind, node_ids, uid=uid)
    #html = build_canvas_html(nodes, edges, level_map, kind, node_ids, uid=uid, viewport_h=320)
    html = build_canvas_html(nodes, edges, level_map, kind, node_ids, uid=uid, viewport_h=320, objetivo_fqn=obj)



    #html = build_svg_html(nodes, edges, level_map, kind, node_ids)

    # tb_linaje
    for s, d, qry in edges:
        tb_linaje_rows.append({
            "tabla_linaje_objetivo": obj,
            "id": node_ids.get(d, ""),
            "tabla_origen": s,
            "tabla_destino": d,
            "query": qry
        })

    tb_html_rows.append({"tabla_destino": obj, "html": html})

tb_linaje_v2 = pd.DataFrame(tb_linaje_rows)
tb_html_v2 = pd.DataFrame(tb_html_rows)

# -----------------------------
# Guardar CSVs
# -----------------------------
out_dir = "./resultados_linaje"
linaje_path = out_dir + "/tb_linaje_generado.csv"
html_path = out_dir +"/tb_html_generado.csv"

tb_linaje_v2.to_csv(linaje_path, index=False, quoting=csv.QUOTE_ALL)
tb_html_v2.to_csv(html_path, index=False, quoting=csv.QUOTE_ALL)

summary = pd.DataFrame({
    "objetivos_destino_upstream": [len(dest_targets)],
    "objetivos_crudos_downstream": [len(raw_sources)],
    "total_objetivos": [len(objectives)],
    "filas_tb_linaje": [len(tb_linaje_v2)],
    "filas_tb_html": [len(tb_html_v2)]
})

(str(linaje_path), str(html_path), summary)

('./resultados_linaje/tb_linaje_generado.csv',
 './resultados_linaje/tb_html_generado.csv',
    objetivos_destino_upstream  objetivos_crudos_downstream  total_objetivos  \
 0                          46                           20               66   
 
    filas_tb_linaje  filas_tb_html  
 0              818             66  )